<a href="https://colab.research.google.com/github/Sriramlingavel/MultiModal_Fashion_LLM/blob/main/MultiModal_FashionLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tiktoken tensorflow bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 131.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

# Replace 'your_file.zip' with the name of the uploaded zip file
with zipfile.ZipFile('/content/drive/MyDrive/gpt2_model.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Files')

In [ ]:
import os
import torch
import torch.nn as nn
import tiktoken
import numpy as np
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ------------------------------
# Custom Model Architecture
# ------------------------------

class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

class GELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) *
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=True):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)

        return context_vec

class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"],
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):
        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x

class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

# ------------------------------
# Data Loading and Preprocessing
# ------------------------------

class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

def create_dataloader(txt, batch_size=4, max_length=1024,
                      stride=512, shuffle=True, drop_last=True,
                      num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDataset(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

# ------------------------------
# GPT-2 Model Setup
# ------------------------------

# Configuration for GPT-2 large (774M)
GPT_CONFIG_774M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 1280,
    "n_heads": 20,
    "n_layers": 36,
    "drop_rate": 0.1,
    "qkv_bias": True
}

# ------------------------------
# GPT-2 Model Weight Loading (from local path)
# ------------------------------

def load_gpt2_params_from_local_path(model_path, model_size="774M"):
    """Load parameters from local GPT-2 model path."""
    try:
        import tensorflow as tf
    except ImportError:
        raise ImportError("TensorFlow is required to load GPT-2 weights.")

    # Define the checkpoint path
    model_dir = os.path.join(model_path, model_size)
    ckpt_path = os.path.join(model_dir, "model.ckpt")
    hparams_path = os.path.join(model_dir, "hparams.json")

    # Check if files exist
    if not os.path.exists(hparams_path):
        raise FileNotFoundError(f"hparams.json not found at {hparams_path}")

    # Load settings
    with open(hparams_path, 'r') as f:
        settings = json.load(f)

    # Initialize params dict
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Load each variable from the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Map TensorFlow variables to our PyTorch model structure
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Build nested dictionary structure
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Store variable in dictionary
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    # Add wte (word token embeddings) and wpe (position embeddings) at top level for convenience
    for name in ["wte", "wpe"]:
        if name in params:
            continue  # Skip if already at top level
        for block in params["blocks"]:
            if name in block:
                params[name] = block[name]
                break

    return settings, params

def load_weights_into_gpt(gpt, params):
    """Load GPT-2 weights from params dict into PyTorch model."""
    # Load embeddings
    gpt.pos_emb.weight = nn.Parameter(torch.tensor(params['wpe']))
    gpt.tok_emb.weight = nn.Parameter(torch.tensor(params['wte']))

    # Load transformer blocks
    for b in range(len(params["blocks"])):
        # Split QKV weights and biases
        block = params["blocks"][b]

        # Handle attention layers
        if "attn" in block:
            # QKV weights
            q_w, k_w, v_w = np.split(block["attn"]["c_attn"]["w"], 3, axis=-1)
            gpt.trf_blocks[b].att.W_query.weight = nn.Parameter(torch.tensor(q_w.T))
            gpt.trf_blocks[b].att.W_key.weight = nn.Parameter(torch.tensor(k_w.T))
            gpt.trf_blocks[b].att.W_value.weight = nn.Parameter(torch.tensor(v_w.T))

            # QKV biases
            q_b, k_b, v_b = np.split(block["attn"]["c_attn"]["b"], 3, axis=-1)
            gpt.trf_blocks[b].att.W_query.bias = nn.Parameter(torch.tensor(q_b))
            gpt.trf_blocks[b].att.W_key.bias = nn.Parameter(torch.tensor(k_b))
            gpt.trf_blocks[b].att.W_value.bias = nn.Parameter(torch.tensor(v_b))

            # Output projection
            gpt.trf_blocks[b].att.out_proj.weight = nn.Parameter(torch.tensor(block["attn"]["c_proj"]["w"].T))
            gpt.trf_blocks[b].att.out_proj.bias = nn.Parameter(torch.tensor(block["attn"]["c_proj"]["b"]))

        # Handle feed-forward layers
        if "mlp" in block:
            # First linear layer
            gpt.trf_blocks[b].ff.layers[0].weight = nn.Parameter(torch.tensor(block["mlp"]["c_fc"]["w"].T))
            gpt.trf_blocks[b].ff.layers[0].bias = nn.Parameter(torch.tensor(block["mlp"]["c_fc"]["b"]))

            # Second linear layer (after activation)
            gpt.trf_blocks[b].ff.layers[2].weight = nn.Parameter(torch.tensor(block["mlp"]["c_proj"]["w"].T))
            gpt.trf_blocks[b].ff.layers[2].bias = nn.Parameter(torch.tensor(block["mlp"]["c_proj"]["b"]))

        # Handle layer norms
        if "ln_1" in block:
            gpt.trf_blocks[b].norm1.scale = nn.Parameter(torch.tensor(block["ln_1"]["g"]))
            gpt.trf_blocks[b].norm1.shift = nn.Parameter(torch.tensor(block["ln_1"]["b"]))

        if "ln_2" in block:
            gpt.trf_blocks[b].norm2.scale = nn.Parameter(torch.tensor(block["ln_2"]["g"]))
            gpt.trf_blocks[b].norm2.shift = nn.Parameter(torch.tensor(block["ln_2"]["b"]))

    # Load final layer norm
    if "ln_f" in params:
        gpt.final_norm.scale = nn.Parameter(torch.tensor(params["ln_f"]["g"]))
        gpt.final_norm.shift = nn.Parameter(torch.tensor(params["ln_f"]["b"]))
    elif "g" in params and "b" in params:
        gpt.final_norm.scale = nn.Parameter(torch.tensor(params["g"]))
        gpt.final_norm.shift = nn.Parameter(torch.tensor(params["b"]))

    # Output head (reuse token embedding weights)
    gpt.out_head.weight = nn.Parameter(torch.tensor(params["wte"]))

# ------------------------------
# Text Generation Functions
# ------------------------------

def generate(model, idx, max_new_tokens, context_size, temperature=0.7, top_k=40, eos_id=None):
    """Generate text with temperature and top-k sampling."""
    model.eval()

    for _ in range(max_new_tokens):
        # Get context for current generation step
        idx_cond = idx[:, -context_size:]

        # Forward pass
        with torch.no_grad():
            logits = model(idx_cond)

        # Focus on the last token
        logits = logits[:, -1, :]

        # Apply top-k filtering
        if top_k is not None:
            v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
            logits[logits < v[:, [-1]]] = float('-inf')

        # Apply temperature
        if temperature > 0:
            logits = logits / temperature
            probs = torch.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            # Greedy sampling
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        # Stop if EOS token is generated
        if eos_id is not None and (idx_next == eos_id).any():
            stop_indices = (idx_next == eos_id).nonzero(as_tuple=True)[0]
            if len(stop_indices) > 0:
                break

        # Append new token to sequence
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

def text_to_token_ids(text, tokenizer=None):
    """Convert text to token IDs."""
    if tokenizer is None:
        tokenizer = tiktoken.get_encoding("gpt2")
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    return torch.tensor(encoded).unsqueeze(0)

def token_ids_to_text(token_ids, tokenizer=None):
    """Convert token IDs to text."""
    if tokenizer is None:
        tokenizer = tiktoken.get_encoding("gpt2")
    flat = token_ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

# ------------------------------
# Training Functions
# ------------------------------

def train_step(model, optimizer, batch, device, criterion=nn.CrossEntropyLoss()):
    """Perform one training step."""
    inputs, targets = batch
    inputs, targets = inputs.to(device), targets.to(device)

    # Forward pass
    logits = model(inputs)

    # Reshape for loss calculation
    b, t, c = logits.shape
    logits = logits.view(b * t, c)
    targets = targets.view(b * t)

    # Calculate loss
    loss = criterion(logits, targets)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, val_loader, device, criterion=nn.CrossEntropyLoss()):
    """Evaluate model on validation data."""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            logits = model(inputs)

            # Reshape for loss calculation
            b, t, c = logits.shape
            logits = logits.view(b * t, c)
            targets = targets.view(b * t)

            # Calculate loss
            loss = criterion(logits, targets)
            total_loss += loss.item()

    avg_loss = total_loss / len(val_loader)
    model.train()
    return avg_loss

def train_model(model, train_loader, val_loader, optimizer, device,
                num_epochs=3, save_path="gpt2_774m_model.pt"):
    """Train the model."""
    model.to(device)
    model.train()

    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_losses = []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            loss = train_step(model, optimizer, batch, device, criterion)
            train_losses.append(loss)
            progress_bar.set_postfix({"train_loss": f"{loss:.4f}"})

        # Validation phase
        val_loss = evaluate(model, val_loader, device, criterion)

        # Print epoch results
        avg_train_loss = sum(train_losses) / len(train_losses)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save({
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "epoch": epoch,
                "val_loss": val_loss
            }, save_path)
            print(f"Saved best model to {save_path}")

    return model

# ------------------------------
# Main Functions
# ------------------------------

def load_pretrained_model_from_path(model_path, model_size="774M", device=None):
    """Load the GPT-2 model from a local path."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    print(f"Using device: {device}")

    # Load GPT-2 weights from local path
    print(f"Loading GPT-2 {model_size} model weights from {model_path}...")
    settings, params = load_gpt2_params_from_local_path(model_path, model_size)

    # Initialize our model
    print("Initializing model...")
    model = GPTModel(GPT_CONFIG_774M)

    # Load weights into our model
    print("Loading weights into model...")
    load_weights_into_gpt(model, params)

    # Move model to device
    model.to(device)
    print("Model loaded successfully!")

    return model

def fine_tune_on_text(model, text_file_path, batch_size=2, num_epochs=3, learning_rate=1e-5):
    """Fine-tune the model on a text file."""
    device = next(model.parameters()).device

    # Load text data
    print(f"Loading text data from {text_file_path}...")
    with open(text_file_path, 'r', encoding='utf-8') as f:
        text_data = f.read()

    # Split into train/val
    train_ratio = 0.9
    split_idx = int(train_ratio * len(text_data))
    train_data = text_data[:split_idx]
    val_data = text_data[split_idx:]

    # Create dataloaders
    print("Creating dataloaders...")
    train_loader = create_dataloader(
        train_data,
        batch_size=batch_size,
        max_length=GPT_CONFIG_774M["context_length"],
        stride=512,
        shuffle=True
    )

    val_loader = create_dataloader(
        val_data,
        batch_size=batch_size,
        max_length=GPT_CONFIG_774M["context_length"],
        stride=512,
        shuffle=False
    )

    # Initialize optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.1)

    # Train the model
    print(f"Starting training for {num_epochs} epochs...")
    model = train_model(
        model,
        train_loader,
        val_loader,
        optimizer,
        device,
        num_epochs=num_epochs
    )

    return model

def generate_text(model, prompt, max_length=50, temperature=0.7, top_k=40):
    """Generate text from a prompt."""
    device = next(model.parameters()).device
    tokenizer = tiktoken.get_encoding("gpt2")

    # Convert prompt to token ids
    input_ids = text_to_token_ids(prompt, tokenizer).to(device)

    # Generate text
    output_ids = generate(
        model,
        input_ids,
        max_new_tokens=max_length,
        context_size=GPT_CONFIG_774M["context_length"],
        temperature=temperature,
        top_k=top_k
    )

    # Convert back to text
    generated_text = token_ids_to_text(output_ids, tokenizer)

    return generated_text

def save_model(model, path="gpt2_774m_model.pt"):
    """Save the model to disk."""
    torch.save({
        "model_state_dict": model.state_dict(),
        "config": GPT_CONFIG_774M
    }, path)
    print(f"Model saved to {path}")

def load_model(path="gpt2_774m_model.pt", device=None):
    """Load a saved model from disk."""
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    checkpoint = torch.load(path, map_location=device)
    model = GPTModel(GPT_CONFIG_774M)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)

    return model

# ------------------------------
# Example Usage in Google Colab with Drive
# ------------------------------

if __name__ == "__main__":
    # Example usage in Google Colab
    import sys
    is_colab = 'google.colab' in sys.modules

    if is_colab:
        #from google.colab import drive
        #print("Running in Google Colab environment")

        # Mount Google Drive
        #print("Mounting Google Drive...")
        #drive.mount('/content/drive')

        # Specify the path to your uploaded model
        model_path = '/content/Files/gpt2_model'  # Adjust path as needed

        # Check if model exists in the specified path
        if not os.path.exists(os.path.join(model_path, '774M')):
            print(f"GPT-2 model not found at {model_path}/774M")
            print("Please make sure you've uploaded the model to this location")
            exit()
    else:
        # Running locally
        print("Running in local environment")
        model_path = './gpt2_model'  # Default local path

    # Load model from the specified path
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load model
    model = load_pretrained_model_from_path(model_path, device=device)

    # Generate sample text
    sample_text = generate_text(model, "The dress which suits you is")
    print("\nGenerated Text:")
    print(sample_text)

Using device: cpu
Using device: cpu
Loading GPT-2 774M model weights from /content/Files/gpt2_model...
Initializing model...
Loading weights into model...
Model loaded successfully!

Generated Text:
The dress which suits you is not the dress which suits me. You may want to wear the dress to a formal event, but it is not necessary for formal occasions.


Some people don't care for formal attire and will wear more casual attire - even to a formal event


In [ ]:
import json
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import tiktoken
from tqdm import tqdm
from functools import partial
import shutil
import zipfile

# Import bitsandbytes for 8-bit quantization
import bitsandbytes as bnb

# Load the fashion dataset
def load_fashion_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    print(f"Loaded {len(data)} entries from {file_path}")
    return data

# Format input as per your requirements
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry.get('input', '') else ""

    return instruction_text + input_text

# Custom Dataset class for instruction tuning
class InstructionDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

        # Pre-tokenize texts
        self.encoded_texts = []
        for entry in data:
            instruction_plus_input = format_input(entry)
            response_text = f"\n\n### Response:\n{entry['output']}"
            full_text = instruction_plus_input + response_text
            self.encoded_texts.append(
                tokenizer.encode(full_text, allowed_special={"<|endoftext|>"})
            )

    def __getitem__(self, index):
        return self.encoded_texts[index]

    def __len__(self):
        return len(self.data)

# Collate function for padding sequences in a batch
def custom_collate_fn(
    batch,
    pad_token_id=50256,  # <|endoftext|> token id for GPT-2
    ignore_index=-100,
    max_length=None,
    device="cuda"
):
    # Find the max length in the batch
    batch_max_length = max(len(item)+1 for item in batch)

    # Limit to max_length if specified
    if max_length is not None and batch_max_length > max_length:
        batch_max_length = max_length

    # Prepare inputs and targets
    inputs_lst, targets_lst = [], []

    for item in batch:
        new_item = item.copy()
        # Add an <|endoftext|> token
        new_item += [pad_token_id]

        # Truncate if longer than max_length
        if max_length is not None:
            new_item = new_item[:max_length+1]  # +1 for the added token

        # Pad sequences to batch_max_length
        padded = new_item + [pad_token_id] * (batch_max_length - len(new_item))

        inputs = torch.tensor(padded[:-1])  # Inputs: all tokens except the last
        targets = torch.tensor(padded[1:])  # Targets: all tokens shifted by 1

        # Replace padding tokens in targets with ignore_index for loss calculation
        mask = targets == pad_token_id
        indices = torch.nonzero(mask).squeeze()
        if indices.numel() > 1:  # If there's more than one padding token
            if isinstance(indices, torch.Tensor) and indices.dim() > 0:
                targets[indices[1:]] = ignore_index  # Keep only the first padding token

        inputs_lst.append(inputs)
        targets_lst.append(targets)

    # Stack tensors
    inputs_tensor = torch.stack(inputs_lst)
    targets_tensor = torch.stack(targets_lst)

    # Use cpu for collation then move to device later - better memory management
    return inputs_tensor.to(device), targets_tensor.to(device)

def quantize_model(model, use_8bit=True):
    """
    Convert the model to 8-bit quantization for both inference and training if GPU is available
    """
    if not use_8bit:
        print("Skipping 8-bit quantization as running on CPU")
        return model

    print("Quantizing model to 8-bit...")

    # Replace linear layers with 8-bit quantized linear layers
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            parent_name = '.'.join(name.split('.')[:-1])
            child_name = name.split('.')[-1]
            parent = model

            if parent_name:
                for part in parent_name.split('.'):
                    parent = getattr(parent, part)

            # Replace with 8-bit quantized linear layer
            setattr(parent, child_name, bnb.nn.Linear8bitLt(
                module.in_features,
                module.out_features,
                module.bias is not None
            ))

            # Copy weights and bias
            if module.bias is not None:
                getattr(parent, child_name).bias = nn.Parameter(module.bias.data)

            with torch.no_grad():
                getattr(parent, child_name).weight.copy_(module.weight)

    print("Model quantized to 8-bit successfully!")
    return model

def setup_model(device, load_from_checkpoint=None, use_8bit=True):
    """Set up the GPT-2 774M model and quantize it to 8-bit if use_8bit is True."""
    print(f"Using device: {device}")

    # Download and load GPT-2 774M weights
    print("Downloading/loading GPT-2 774M model weights...")
    settings, params = load_gpt2_params_from_local_path(model_size="774M", model_path='/content/Files/gpt2_model')

    # Initialize model
    print("Initializing model...")
    model = GPTModel(GPT_CONFIG_774M)

    # Load weights
    print("Loading weights into model...")
    load_weights_into_gpt(model, params)

    # Load from checkpoint if provided
    if load_from_checkpoint and os.path.exists(load_from_checkpoint):
        print(f"Loading from checkpoint: {load_from_checkpoint}")
        checkpoint = torch.load(load_from_checkpoint, map_location=device)
        model.load_state_dict(checkpoint["model_state_dict"])
        print(f"Loaded checkpoint successfully!")

    # Move model to device first (important for CPU mode)
    model.to(device)

    # Quantize model to 8-bit if use_8bit is True (requires GPU)
    if use_8bit:
        model = quantize_model(model, use_8bit)
        print("Model quantized successfully!")
    else:
        print("Using full precision model (no quantization)")

    return model

def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    device,
    num_epochs=3,
    eval_freq=100,  # Evaluate every n steps
    save_dir="checkpoints",
    output_dir="fashion_files"  # Changed path
):
    """Train the model on fashion dataset."""
    model.to(device)
    model.train()

    # Create dirs if they don't exist
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    # For tracking
    best_val_loss = float('inf')
    best_epoch = -1
    global_step = 0
    train_losses = []
    val_losses = []

    # Path to save the best model
    best_model_path = os.path.join(save_dir, "best_model.pt")

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        epoch_losses = []

        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (inputs, targets) in enumerate(progress_bar):
            # Forward pass
            logits = model(inputs)

            # Calculate loss (reshape for cross entropy)
            b, t, c = logits.shape
            loss = torch.nn.functional.cross_entropy(
                logits.view(b*t, c),
                targets.view(b*t)
            )

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Record loss
            loss_value = loss.item()
            epoch_losses.append(loss_value)
            train_losses.append(loss_value)

            # Update progress bar
            progress_bar.set_postfix({"train_loss": f"{loss_value:.4f}"})

            # Evaluate periodically
            global_step += 1
            if global_step % eval_freq == 0:
                val_loss = evaluate_model(model, val_loader, device)
                val_losses.append(val_loss)
                print(f"Step {global_step}: Val Loss = {val_loss:.4f}")

                # Save if best
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    best_epoch = epoch
                    torch.save({
                        "model_state_dict": model.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "epoch": epoch,
                        "global_step": global_step,
                        "val_loss": val_loss
                    }, best_model_path)
                    print(f"Saved best model (val_loss: {val_loss:.4f}) to {best_model_path}")

                # Switch back to training mode
                model.train()

        # End of epoch
        avg_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print(f"Epoch {epoch+1}/{num_epochs} completed. Avg train loss: {avg_epoch_loss:.4f}")

        # Final evaluation for this epoch
        val_loss = evaluate_model(model, val_loader, device)
        print(f"Epoch {epoch+1}/{num_epochs}: Val Loss = {val_loss:.4f}")

        # Save checkpoint at end of epoch
        epoch_save_path = os.path.join(save_dir, f"fashion_gpt2_epoch{epoch+1}.pt")
        torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "epoch": epoch,
            "global_step": global_step,
            "val_loss": val_loss
        }, epoch_save_path)

        # Update best model if needed
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_epoch = epoch
            shutil.copy(epoch_save_path, best_model_path)
            print(f"Updated best model with val_loss: {val_loss:.4f}")

    # After all epochs, zip and save the best model
    print(f"Best model was from epoch {best_epoch+1} with val_loss: {best_val_loss:.4f}")

    best_model_zip = os.path.join(output_dir, "best_model.zip")
    with zipfile.ZipFile(best_model_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(best_model_path, arcname=os.path.basename(best_model_path))

    print(f"Best model zipped and saved to {best_model_zip}")

    return model, train_losses, val_losses, best_epoch+1

def evaluate_model(model, data_loader, device, num_batches=None):
    """Evaluate the model on validation data."""
    model.eval()
    total_loss = 0
    batch_count = 0

    # Determine how many batches to evaluate
    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))

    with torch.no_grad():
        for i, (inputs, targets) in enumerate(data_loader):
            if i >= num_batches:
                break

            # Forward pass
            logits = model(inputs)

            # Calculate loss
            b, t, c = logits.shape
            loss = torch.nn.functional.cross_entropy(
                logits.view(b*t, c),
                targets.view(b*t)
            )

            total_loss += loss.item()
            batch_count += 1

    return total_loss / batch_count if batch_count > 0 else float('inf')

def generate_samples(model, tokenizer, val_data, device, num_samples=3):
    """Generate text samples using the model."""
    model.eval()
    context_size = model.pos_emb.weight.shape[0]

    for i in range(min(num_samples, len(val_data))):
        entry = val_data[i]
        prompt = format_input(entry)

        # Tokenize the prompt
        encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)

        # Generate response
        with torch.no_grad():
            token_ids = generate_text(
                model=model,
                idx=encoded,
                max_new_tokens=150,
                context_size=context_size,
                temperature=0.7,
                top_k=40
            )

        # Decode the generated text
        generated_text = tokenizer.decode(token_ids.squeeze().tolist())

        # Print results
        print(f"\n--- Sample {i+1} ---")
        print(f"Prompt: {prompt}")
        print(f"Generated: {generated_text[len(prompt):]}")
        print(f"Expected: {entry['output']}")
        print("-" * 40)

def generate_text(model, idx, max_new_tokens, context_size, temperature=0.7, top_k=40, eos_id=50256):
    """Generate text with the model."""
    model.eval()

    for _ in range(max_new_tokens):
        # Get context for the current step
        idx_cond = idx[:, -context_size:]

        # Forward pass
        with torch.no_grad():
            logits = model(idx_cond)

        # Get logits for the last token
        logits = logits[:, -1, :]

        # Apply temperature and top-k filtering
        if temperature > 0:
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = float('-inf')

            probs = torch.softmax(logits / temperature, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
        else:
            # Greedy sampling
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)

        # Append new token
        idx = torch.cat((idx, idx_next), dim=1)

        # Stop if EOS token is generated
        if idx_next.item() == eos_id:
            break

    return idx

def main():
    # Check if GPU is available - bitsandbytes requires GPU
    if not torch.cuda.is_available():
        print("WARNING: GPU not available. bitsandbytes requires GPU.")
        print("Switching to standard training without 8-bit quantization.")
        use_8bit = False
        device = torch.device("cpu")
    else:
        use_8bit = True
        device = torch.device("cuda")

    print(f"Using device: {device}")
    print(f"Using 8-bit quantization: {use_8bit}")

    # Mount Google Drive if running in Colab
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_mounted = True
        print("Google Drive mounted successfully")
    except ImportError:
        drive_mounted = False
        print("Not running in Colab or Drive already mounted")

    # 1. Load dataset - update path if using Google Drive
    dataset_path = "/content/drive/MyDrive/recommendation_fashion_dataset.json" if drive_mounted else "men_fashion_dataset.json"
    data = load_fashion_dataset(dataset_path)

    # 2. Split data
    train_ratio = 0.85
    val_ratio = 0.15

    total_size = len(data)
    train_size = int(total_size * train_ratio)

    # Shuffle data (optional)
    import random
    random.seed(42)
    random.shuffle(data)

    train_data = data[:train_size]
    val_data = data[train_size:]

    print(f"Training set size: {len(train_data)}")
    print(f"Validation set size: {len(val_data)}")

    # 3. Setup tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # 4. Create datasets and dataloaders
    train_dataset = InstructionDataset(train_data, tokenizer)
    val_dataset = InstructionDataset(val_data, tokenizer)

    batch_size = 2  # Adjust based on your GPU memory
    max_length = 1024  # Context length

    # Customize collate function with your device
    # If on CPU, keep tensors on CPU during collation for memory efficiency
    collate_device = "cpu" if device.type == "cpu" else device
    collate_fn = partial(
        custom_collate_fn,
        device=collate_device,
        max_length=max_length
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0  # Adjust based on your environment
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=0
    )

    # 5. Check for existing best model - fix paths
    base_dir = "/content/drive/MyDrive/fashion_files" if drive_mounted else "fashion_files"
    os.makedirs(base_dir, exist_ok=True)
    best_model_path = os.path.join(base_dir, "best_model.zip")
    checkpoint_path = None

    if os.path.exists(best_model_path):
        print(f"Found existing best model: {best_model_path}")
        # Extract best model for continued training
        with zipfile.ZipFile(best_model_path, 'r') as zip_ref:
            zip_ref.extractall("temp_model")

        checkpoint_path = "temp_model/best_model.pt"
        print(f"Extracted checkpoint to: {checkpoint_path}")

    # 6. Setup model with optional 8-bit quantization
    model = setup_model(device, load_from_checkpoint=checkpoint_path, use_8bit=use_8bit)

    # 7. Setup optimizer (with 8-bit optimization if using GPU)
    if use_8bit:
        optimizer = bnb.optim.AdamW8bit(
            model.parameters(),
            lr=5e-5,  # Learning rate
            weight_decay=0.1  # L2 regularization
        )
    else:
        # Use standard AdamW for CPU
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=5e-5,  # Learning rate
            weight_decay=0.1  # L2 regularization
        )

    # 8. Fine-tune the model
    print(f"Starting fine-tuning {'with 8-bit quantization' if use_8bit else 'in full precision mode'}...")
    model, train_losses, val_losses, best_epoch = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        optimizer=optimizer,
        device=device,
        num_epochs=3,  # Adjust based on your needs
        eval_freq=50,  # Evaluate every 50 steps
        save_dir="checkpoints",
        output_dir=base_dir
    )

    # 9. Generate some samples to evaluate the model
    print(f"\nGenerating samples using the best model (Epoch {best_epoch})...")
    # Load the best model for generation
    best_model = setup_model(device, use_8bit=use_8bit)
    best_model_checkpoint = torch.load("checkpoints/best_model.pt", map_location=device)
    best_model.load_state_dict(best_model_checkpoint["model_state_dict"])

    generate_samples(best_model, tokenizer, val_data, device, num_samples=3)

    print(f"Fine-tuning completed! Best model from epoch {best_epoch} saved to {os.path.join(base_dir, 'best_model.zip')}")

if __name__ == "__main__":
    main()

Switching to standard training without 8-bit quantization.
Using device: cpu
Using 8-bit quantization: False


KeyboardInterrupt: 

In [ ]:
import os
import zipfile
import shutil
from tqdm import tqdm

# Define source and destination paths
source_model_path = '/content/checkpoints/best_model.pt'
destination_folder = '/content/drive/MyDrive/fashion_files/'
zip_filename = 'best_model.zip'
zip_filepath = os.path.join(destination_folder, zip_filename)

# Ensure the destination directory exists
os.makedirs(destination_folder, exist_ok=True)

# Get file size for tqdm
file_size = os.path.getsize(source_model_path)

# Custom class to track progress with tqdm
class TqdmZipFile(zipfile.ZipFile):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def write(self, filename, arcname=None, compress_type=None):
        if arcname is None:
            arcname = os.path.basename(filename)

        with open(filename, 'rb') as f:
            # Set up tqdm progress bar
            with tqdm(total=os.path.getsize(filename),
                      unit='B',
                      unit_scale=True,
                      desc=f"Zipping {os.path.basename(filename)}") as pbar:
                # Create a custom file-like object to update the progress bar
                class TqdmReadFile:
                    def __init__(self, file, pbar):
                        self.file = file
                        self.pbar = pbar

                    def read(self, size):
                        data = self.file.read(size)
                        self.pbar.update(len(data))
                        return data

                zinfo = zipfile.ZipInfo.from_file(filename, arcname)
                if compress_type is not None:
                    zinfo.compress_type = compress_type
                else:
                    zinfo.compress_type = self.compression

                with self.open(zinfo, 'w') as dest:
                    shutil.copyfileobj(TqdmReadFile(f, pbar), dest)

# Create zip file with progress tracking
try:
    print(f"Creating zip file: {zip_filepath}")
    with TqdmZipFile(zip_filepath, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(source_model_path, arcname=os.path.basename(source_model_path))
    print(f"Successfully created zip at: {zip_filepath}")

except Exception as e:
    print(f"Error during zipping: {str(e)}")

Creating zip file: /content/drive/MyDrive/fashion_files/best_model.zip


Zipping best_model.pt: 100%|██████████| 10.2G/10.2G [08:42<00:00, 19.6MB/s]

Successfully created zip at: /content/drive/MyDrive/fashion_files/best_model.zip


In [ ]:
import os
import zipfile
import torch
import tiktoken
from tqdm import tqdm

# Define paths
zip_filepath = '/content/drive/MyDrive/fashion_files/best_model.zip'
unzip_destination = '/content/extracted_model/'

# Ensure directory exists
os.makedirs(unzip_destination, exist_ok=True)

# Function to extract zip with progress tracking
def extract_with_progress(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get file info
        file_info_list = zip_ref.infolist()

        # Set up tqdm progress bar
        with tqdm(total=len(file_info_list), desc="Extracting files") as pbar:
            for file_info in file_info_list:
                zip_ref.extract(file_info, extract_to)
                pbar.update(1)

    print(f"Extraction complete to {extract_to}")
    return os.path.join(extract_to, os.path.basename(zip_path).replace('.zip', '.pt'))

# Model loading and interaction functions
def format_input(instruction, input_text=""):
    """Format the instruction and input according to the training format."""
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request."
        f"\n\n### Instruction:\n{instruction}"
    )
    input_part = f"\n\n### Input:\n{input_text}" if input_text else ""
    return instruction_text + input_part

def load_finetuned_model(checkpoint_path, device):
    """Load the fine-tuned model from a checkpoint."""
    print(f"Loading model from {checkpoint_path}")

    # Import your model definitions here
    #from gpt_model import GPTModel, GPT_CONFIG_774M

    # Initialize model with configuration
    model = GPTModel(GPT_CONFIG_774M)

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])

    # Move model to device
    model.to(device)
    model.eval()

    print(f"Model loaded successfully! Validation loss: {checkpoint.get('val_loss', 'N/A')}")
    return model

def generate_text(model, tokenizer, prompt, device, max_new_tokens=500, temperature=0.7, top_k=40):
    """Generate text based on a prompt using the fine-tuned model."""
    # Tokenize the prompt
    encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)
    context_size = model.pos_emb.weight.shape[0]

    # Generate response
    model.eval()

    with torch.no_grad():
        for _ in range(max_new_tokens):
            # Get context for the current step
            idx_cond = encoded[:, -context_size:]

            # Forward pass
            logits = model(idx_cond)

            # Get logits for the last token
            logits = logits[:, -1, :]

            # Apply temperature and top-k filtering
            if temperature > 0:
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = float('-inf')

                probs = torch.softmax(logits / temperature, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)

            # Append new token
            encoded = torch.cat((encoded, idx_next), dim=1)

            # Stop if EOS token is generated (50256 is <|endoftext|>)
            if idx_next.item() == 50256:
                break

    # Decode the generated text
    generated_text = tokenizer.decode(encoded.squeeze().tolist())

    # Return only the response part (after the prompt)
    return generated_text[len(prompt):]

def test_model_interactive(model, tokenizer, device):
    """Interactive testing of the model with custom prompts."""
    print("\n===== Interactive Testing Mode =====")
    print("Enter your queries to test the model. Type 'exit' to quit.")
    print("Format: instruction | input (optional)")
    print("\nSuggested fashion-related queries:")
    print("1. Describe this outfit: | A black leather jacket with white t-shirt and blue jeans")
    print("2. Suggest accessories for | a summer wedding guest outfit with a floral dress")
    print("3. Compare these styles: | minimalist vs maximalist fashion")
    print("4. Create a shopping list for | building a capsule wardrobe for fall")
    print("5. What are current trends in | sustainable fashion for 2024?")

    while True:
        user_input = input("\nYour query: ")

        if user_input.lower() == 'exit':
            break

        # Parse instruction and optional input
        parts = user_input.split('|', 1)
        instruction = parts[0].strip()
        input_text = parts[1].strip() if len(parts) > 1 else ""

        # Format the prompt
        prompt = format_input(instruction, input_text)

        print("\nGenerating response...")
        response = generate_text(model, tokenizer, prompt, device)

        print("\n--- Result ---")
        print(f"Prompt: {prompt}")
        print(f"Generated response: {response}")
        print("---------------")

def main():
    # Unzip the model file
    print("Unzipping model file...")
    extracted_model_path = extract_with_progress(zip_filepath, unzip_destination)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Load the fine-tuned model
    model = load_finetuned_model(extracted_model_path, device)

    # Test model interactively
    test_model_interactive(model, tokenizer, device)

if __name__ == "__main__":
    main()

Unzipping model file...


Extracting files: 100%|██████████| 1/1 [01:34<00:00, 94.50s/it]


Extraction complete to /content/extracted_model/
Using device: cpu
Loading model from /content/extracted_model/best_model.pt
Model loaded successfully! Validation loss: 1.2289036760727565

===== Interactive Testing Mode =====
Enter your queries to test the model. Type 'exit' to quit.
Format: instruction | input (optional)

Suggested fashion-related queries:
1. Describe this outfit: | A black leather jacket with white t-shirt and blue jeans
2. Suggest accessories for | a summer wedding guest outfit with a floral dress
3. Compare these styles: | minimalist vs maximalist fashion
4. Create a shopping list for | building a capsule wardrobe for fall
5. What are current trends in | sustainable fashion for 2024?

Your query: suggest a pants for pink polo t shirt

Generating response...

--- Result ---
Prompt: Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
suggest a pants for pink polo t shirt
Generated response: .  I'

KeyboardInterrupt: Interrupted by user

In [ ]:
import os
import zipfile
import torch
import tiktoken
import json
from tqdm import tqdm

# Define paths
zip_filepath = '/content/drive/MyDrive/fashion_files/best_model.zip'
unzip_destination = '/content/extracted_model/'
user_text_path = '/content/drive/MyDrive/fashion_files/user.txt'
config_path = '/content/drive/MyDrive/fashion_files/generation_config.json'

# Default generation parameters (will be overridden by config file if present)
DEFAULT_CONFIG = {
    "max_new_tokens": 150,
    "temperature": 0.7,
    "top_k": 40,
    "instruction": "",
    "input_text": ""
}

# Ensure directory exists
os.makedirs(unzip_destination, exist_ok=True)

# Function to extract zip with progress tracking
def extract_with_progress(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get file info
        file_info_list = zip_ref.infolist()

        # Set up tqdm progress bar
        with tqdm(total=len(file_info_list), desc="Extracting files") as pbar:
            for file_info in file_info_list:
                zip_ref.extract(file_info, extract_to)
                pbar.update(1)

    print(f"Extraction complete to {extract_to}")
    return os.path.join(extract_to, os.path.basename(zip_path).replace('.zip', '.pt'))

# Model loading and interaction functions
def format_input(instruction, input_text=""):
    """Format the instruction and input according to the training format."""
    # Modified to explicitly tell the model not to generate additional inputs
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request. "
        f"Do not generate additional inputs or profile information."
        f"\n\n### Instruction:\n{instruction}"
    )
    input_part = f"\n\n### Input:\n{input_text}" if input_text else ""
    return instruction_text + input_part + "\n\n### Response:"

def load_finetuned_model(checkpoint_path, device):
    """Load the fine-tuned model from a checkpoint."""
    print(f"Loading model from {checkpoint_path}")

    # Import your model definitions here
    #from gpt_model import GPTModel, GPT_CONFIG_774M

    # Initialize model with configuration
    model = GPTModel(GPT_CONFIG_774M)

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])

    # Move model to device
    model.to(device)
    model.eval()

    print(f"Model loaded successfully! Validation loss: {checkpoint.get('val_loss', 'N/A')}")
    return model

def generate_text(model, tokenizer, prompt, device, max_new_tokens=150, temperature=0.7, top_k=40):
    """Generate text based on a prompt using the fine-tuned model."""
    # Tokenize the prompt
    encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)
    context_size = model.pos_emb.weight.shape[0]

    # Generate response
    model.eval()

    print(f"Generation parameters: max_tokens={max_new_tokens}, temp={temperature}, top_k={top_k}")
    generated_tokens = []

    with torch.no_grad():
        for _ in tqdm(range(max_new_tokens), desc="Generating tokens"):
            # Get context for the current step
            idx_cond = encoded[:, -context_size:]

            # Forward pass
            logits = model(idx_cond)

            # Get logits for the last token
            logits = logits[:, -1, :]

            # Apply temperature and top-k filtering
            if temperature > 0:
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = float('-inf')

                probs = torch.softmax(logits / temperature, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)

            # Append new token
            encoded = torch.cat((encoded, idx_next), dim=1)
            generated_tokens.append(idx_next.item())

            # Stop if EOS token is generated (50256 is <|endoftext|>)
            if idx_next.item() == 50256:
                break

            # Also stop if we see "### Input:" - this is a sign the model is hallucinating additional input
            current_text = tokenizer.decode(encoded.squeeze().tolist()[len(tokenizer.encode(prompt)):])
            if "### Input:" in current_text:
                break

    # Decode the generated text
    generated_text = tokenizer.decode(encoded.squeeze().tolist())

    # Return only the response part (after the prompt)
    response = generated_text[len(prompt):]

    # Remove any hallucinated input sections
    if "### Input:" in response:
        response = response.split("### Input:")[0].strip()

    # Remove any hallucinated response markers
    if "### Response:" in response:
        response = response.replace("### Response:", "").strip()

    return response, generated_tokens

def load_user_config():
    """Load user configuration from config file if it exists, otherwise use defaults."""
    config = DEFAULT_CONFIG.copy()

    # Check if config file exists
    if os.path.exists(config_path):
        try:
            with open(config_path, 'r') as f:
                user_config = json.load(f)
                config.update(user_config)
                print(f"Loaded configuration from {config_path}")
        except Exception as e:
            print(f"Error loading config file: {e}")
            print("Using default configuration")
    else:
        print(f"Config file not found at {config_path}. Using default configuration.")
        # Create a template config file for future use
        try:
            with open(config_path, 'w') as f:
                json.dump(DEFAULT_CONFIG, f, indent=4)
                print(f"Created template config file at {config_path}")
        except Exception as e:
            print(f"Failed to create template config file: {e}")

    return config

def load_user_text():
    """Load user text from file if it exists."""
    if os.path.exists(user_text_path):
        try:
            with open(user_text_path, 'r') as f:
                content = f.read().strip()
                print(f"Loaded user text from {user_text_path} ({len(content)} characters)")
                return content
        except Exception as e:
            print(f"Error loading user text file: {e}")
            return None
    else:
        print(f"User text file not found at {user_text_path}")
        return None

def update_config_interactive(config):
    """Allow user to interactively update configuration parameters."""
    print("\n===== Generation Configuration =====")
    print("Current settings:")
    for key, value in config.items():
        print(f"  {key}: {value}")

    print("\nEnter new values (press Enter to keep current value):")

    # Update numerical parameters
    for param in ["max_new_tokens", "temperature", "top_k"]:
        new_value = input(f"{param} [{config[param]}]: ")
        if new_value.strip():
            try:
                if param == "max_new_tokens" or param == "top_k":
                    config[param] = int(new_value)
                else:
                    config[param] = float(new_value)
            except ValueError:
                print(f"Invalid value for {param}, keeping current value.")

    # Save updated config
    try:
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=4)
            print(f"Configuration saved to {config_path}")
    except Exception as e:
        print(f"Failed to save configuration: {e}")

    return config

def save_user_text(text):
    """Save user text to file."""
    try:
        with open(user_text_path, 'w') as f:
            f.write(text)
        print(f"Text saved to {user_text_path}")
    except Exception as e:
        print(f"Failed to save text: {e}")

def process_user_input_file(model, tokenizer, device):
    """Process user input from file and generate response."""
    # Load configuration
    config = load_user_config()

    while True:
        # Get user text
        user_text = load_user_text()
        if user_text is None:
            print("Would you like to create a user.txt file? (y/n)")
            if input().lower() == 'y':
                print("Enter your text (end with a newline and Ctrl+D):")
                text = []
                try:
                    while True:
                        line = input()
                        text.append(line)
                except EOFError:
                    user_text = '\n'.join(text)
                    save_user_text(user_text)
            else:
                print("Switching to interactive mode...")
                return test_model_interactive(model, tokenizer, device)

        # Parse instruction and input from user text
        parts = user_text.split('|', 1)
        config["instruction"] = parts[0].strip()
        config["input_text"] = parts[1].strip() if len(parts) > 1 else ""

        # Allow user to adjust generation parameters
        print("\nWould you like to adjust generation parameters? (y/n)")
        if input().lower() == 'y':
            config = update_config_interactive(config)

        # Format the prompt
        prompt = format_input(config["instruction"], config["input_text"])

        # Generate response
        print("\nGenerating response...")
        response, tokens = generate_text(
            model, tokenizer, prompt, device,
            max_new_tokens=config["max_new_tokens"],
            temperature=config["temperature"],
            top_k=config["top_k"]
        )

        # Display results
        print("\n=== Generated Response ===")
        print(response)
        print("\n=== Token Stats ===")
        print(f"Generated {len(tokens)} tokens")
        print(f"Input tokens: {len(tokenizer.encode(prompt))}")
        print(f"Total tokens: {len(tokenizer.encode(prompt)) + len(tokens)}")

        # Save response
        response_path = os.path.join(os.path.dirname(user_text_path), "response.txt")
        try:
            with open(response_path, 'w') as f:
                f.write(response)
            print(f"Response saved to {response_path}")
        except Exception as e:
            print(f"Failed to save response: {e}")

        # Ask to continue
        print("\nOptions:")
        print("1. Process the same text with different parameters")
        print("2. Edit the user.txt file")
        print("3. Switch to interactive mode")
        print("4. Exit")
        choice = input("Choose an option (1-4): ")

        if choice == '1':
            continue
        elif choice == '2':
            print("Enter new text (end with a newline and Ctrl+D):")
            text = []
            try:
                while True:
                    line = input()
                    text.append(line)
            except EOFError:
                user_text = '\n'.join(text)
                save_user_text(user_text)
        elif choice == '3':
            return test_model_interactive(model, tokenizer, device)
        else:
            break

def test_model_interactive(model, tokenizer, device):
    """Interactive testing of the model with custom prompts."""
    print("\n===== Interactive Testing Mode =====")
    print("Enter your queries to test the model. Type 'exit' to quit.")
    print("Format: instruction | input (optional)")

    # Provide helpful suggestions
    print("\nSuggested fashion-related queries:")
    print("1. Describe this outfit: | A black leather jacket with white t-shirt and blue jeans")
    print("2. Suggest accessories for | a summer wedding guest outfit with a floral dress")
    print("3. Compare these styles: | minimalist vs maximalist fashion")
    print("4. Create a shopping list for | building a capsule wardrobe for fall")
    print("5. What are current trends in | sustainable fashion for 2024?")

    # Load default config for generation parameters
    config = load_user_config()

    while True:
        # Ask to adjust generation parameters
        print("\nWould you like to adjust generation parameters? (y/n)")
        if input().lower() == 'y':
            config = update_config_interactive(config)

        user_input = input("\nYour query: ")

        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'file':
            return process_user_input_file(model, tokenizer, device)

        # Parse instruction and optional input
        parts = user_input.split('|', 1)
        instruction = parts[0].strip()
        input_text = parts[1].strip() if len(parts) > 1 else ""

        # Format the prompt
        prompt = format_input(instruction, input_text)

        print("\nGenerating response...")
        response, tokens = generate_text(
            model, tokenizer, prompt, device,
            max_new_tokens=config["max_new_tokens"],
            temperature=config["temperature"],
            top_k=config["top_k"]
        )

        print("\n=== Generated Response ===")
        print(response)
        print("\n=== Token Stats ===")
        print(f"Generated {len(tokens)} tokens")

def main():
    # Unzip the model file
    print("Unzipping model file...")
    extracted_model_path = extract_with_progress(zip_filepath, unzip_destination)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Load the fine-tuned model
    model = load_finetuned_model(extracted_model_path, device)

    # Check if user.txt file exists
    if os.path.exists(user_text_path):
        print(f"Found user text file at {user_text_path}")
        process_user_input_file(model, tokenizer, device)
    else:
        print("No user text file found. Starting in interactive mode.")
        # Test model interactively
        test_model_interactive(model, tokenizer, device)

if __name__ == "__main__":
    main()

Unzipping model file...


Extracting files: 100%|██████████| 1/1 [01:26<00:00, 86.94s/it]


Extraction complete to /content/extracted_model/
Using device: cpu
Loading model from /content/extracted_model/best_model.pt
Model loaded successfully! Validation loss: 1.2289036760727565
No user text file found. Starting in interactive mode.

===== Interactive Testing Mode =====
Enter your queries to test the model. Type 'exit' to quit.
Format: instruction | input (optional)

Suggested fashion-related queries:
1. Describe this outfit: | A black leather jacket with white t-shirt and blue jeans
2. Suggest accessories for | a summer wedding guest outfit with a floral dress
3. Compare these styles: | minimalist vs maximalist fashion
4. Create a shopping list for | building a capsule wardrobe for fall
5. What are current trends in | sustainable fashion for 2024?
Config file not found at /content/drive/MyDrive/fashion_files/generation_config.json. Using default configuration.
Created template config file at /content/drive/MyDrive/fashion_files/generation_config.json

Would you like to adjus

Generating tokens: 100%|██████████| 300/300 [07:50<00:00,  1.57s/it]



=== Generated Response ===

Hey there! Pants for a Pink Polo Tshirt are perfect!  While your existing t-shirt is a bit too casual, we can definitely elevate it with some fresh pants.

I recommend a pair of well-fitting dark wash jeans or chinos.  Dark wash denim will create a sleek, polished look, ideal for balancing a bold polo.  Make sure the length hits just above the knee, avoiding anything too baggy.  For a subtle pop of pink, consider a subtle pink bandana tied loosely around the neck or a pocket square with a pink pendant.  This adds a feminine touch without being overwhelming.

For a slightly more relaxed vibe, consider a pair of dark wash denim shorts or cargo shorts that hit just above the knee.  Cargo shorts offer a slightly more structured feel than denim shorts but are still comfortable.  

Accessorize with a simple silver chain necklace or a sleek, textured pocket square.  A brown leather belt will complement the tan pants nicely.  Since you're tall, a good knee-length w

In [ ]:
import os
import zipfile
import torch
import tiktoken
import json
from tqdm import tqdm

# Define paths
zip_filepath = '/content/drive/MyDrive/fashion_files/best_model.zip'
unzip_destination = '/content/extracted_model/'
user_text_path = '/content/drive/MyDrive/fashion_files/user.txt'
config_path = '/content/drive/MyDrive/fashion_files/generation_config.json'

# Default generation parameters (will be overridden by config file if present)
DEFAULT_CONFIG = {
    "max_new_tokens": 150,
    "temperature": 0.7,
    "top_k": 40,
    "instruction": "",
    "input_text": "",
    "consider_wardrobe": "no",  # New parameter for personalization
    "consider_user_details": "no"  # New parameter for personalization
}

# Ensure directory exists
os.makedirs(unzip_destination, exist_ok=True)

# Function to extract zip with progress tracking
def extract_with_progress(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get file info
        file_info_list = zip_ref.infolist()

        # Set up tqdm progress bar
        with tqdm(total=len(file_info_list), desc="Extracting files") as pbar:
            for file_info in file_info_list:
                zip_ref.extract(file_info, extract_to)
                pbar.update(1)

    print(f"Extraction complete to {extract_to}")
    return os.path.join(extract_to, os.path.basename(zip_path).replace('.zip', '.pt'))

# New class for parsing and handling user profile data
class UserProfileManager:
    def __init__(self, user_text_path):
        self.user_text_path = user_text_path
        self.user_data = {}
        self.user_wardrobe = []
        self.load_user_data()

    def load_user_data(self):
        """Load and parse user data from user.txt"""
        if not os.path.exists(self.user_text_path):
            print(f"User profile not found at {self.user_text_path}")
            return False

        try:
            with open(self.user_text_path, 'r') as f:
                content = f.read().strip()
                # Parse the user data
                self._parse_user_data(content)
                print(f"Loaded user profile from {self.user_text_path}")
                return True
        except Exception as e:
            print(f"Error loading user profile: {e}")
            return False

    def _parse_user_data(self, content):
        """Parse user data content into structured data"""
        lines = content.split('\n')
        for line in lines:
            # Skip empty lines and lines without a proper key:value format
            if not line.strip() or ':' not in line:
                continue

            parts = line.split(':', 1)
            key = parts[0].strip().lower()
            value = parts[1].strip()

            # Special handling for wardrobe items
            if key == 'wardrobe':
                self.user_wardrobe = [item.strip() for item in value.split(',')]
                self.user_data[key] = self.user_wardrobe
            else:
                self.user_data[key] = value

    def get_user_profile_summary(self):
        """Get a formatted summary of the user profile for the model"""
        if not self.user_data:
            return "No user profile data available."

        summary = []
        # Add basic user information
        for key, value in self.user_data.items():
            if key != 'wardrobe':  # Handle wardrobe separately
                summary.append(f"{key}: {value}")

        return "\n".join(summary)

    def get_wardrobe_summary(self):
        """Get a formatted summary of the user's wardrobe"""
        if not self.user_wardrobe:
            return "No wardrobe information available."

        return "Wardrobe items: " + ", ".join(self.user_wardrobe)

# Model loading and interaction functions
def format_input(instruction, input_text="", consider_wardrobe="no", consider_user_details="no", user_profile=None):
    """Format the instruction and input according to the training format, including personalization options."""
    # Base instruction
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request. "
        f"Do not generate additional inputs or profile information."
        f"\n\n### Instruction:\n{instruction}"
    )

    # Add personalization details if requested
    if user_profile and consider_user_details.lower() == "yes":
        instruction_text += f"\n\nConsider the following user details for personalization:\n{user_profile.get_user_profile_summary()}"

    # Add wardrobe information if requested
    if user_profile and consider_wardrobe.lower() == "yes":
        instruction_text += f"\n\nConsider these items from the user's wardrobe for your suggestions:\n{user_profile.get_wardrobe_summary()}"

    # Add input text if provided
    input_part = f"\n\n### Input:\n{input_text}" if input_text else ""

    return instruction_text + input_part + "\n\n### Response:"

def load_finetuned_model(checkpoint_path, device):
    """Load the fine-tuned model from a checkpoint."""
    print(f"Loading model from {checkpoint_path}")

    # Import your model definitions here
    #from gpt_model import GPTModel, GPT_CONFIG_774M

    # Initialize model with configuration
    model = GPTModel(GPT_CONFIG_774M)

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])

    # Move model to device
    model.to(device)
    model.eval()

    print(f"Model loaded successfully! Validation loss: {checkpoint.get('val_loss', 'N/A')}")
    return model

def generate_text(model, tokenizer, prompt, device, max_new_tokens=150, temperature=0.7, top_k=40):
    """Generate text based on a prompt using the fine-tuned model."""
    # Tokenize the prompt
    encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)
    context_size = model.pos_emb.weight.shape[0]

    # Generate response
    model.eval()

    print(f"Generation parameters: max_tokens={max_new_tokens}, temp={temperature}, top_k={top_k}")
    generated_tokens = []

    with torch.no_grad():
        for _ in tqdm(range(max_new_tokens), desc="Generating tokens"):
            # Get context for the current step
            idx_cond = encoded[:, -context_size:]

            # Forward pass
            logits = model(idx_cond)

            # Get logits for the last token
            logits = logits[:, -1, :]

            # Apply temperature and top-k filtering
            if temperature > 0:
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = float('-inf')

                probs = torch.softmax(logits / temperature, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)

            # Append new token
            encoded = torch.cat((encoded, idx_next), dim=1)
            generated_tokens.append(idx_next.item())

            # Stop if EOS token is generated (50256 is <|endoftext|>)
            if idx_next.item() == 50256:
                break

            # Also stop if we see "### Input:" - this is a sign the model is hallucinating additional input
            current_text = tokenizer.decode(encoded.squeeze().tolist()[len(tokenizer.encode(prompt)):])
            if "### Input:" in current_text:
                break

    # Decode the generated text
    generated_text = tokenizer.decode(encoded.squeeze().tolist())

    # Return only the response part (after the prompt)
    response = generated_text[len(prompt):]

    # Remove any hallucinated input sections
    if "### Input:" in response:
        response = response.split("### Input:")[0].strip()

    # Remove any hallucinated response markers
    if "### Response:" in response:
        response = response.replace("### Response:", "").strip()

    return response, generated_tokens

def load_user_config():
    """Load user configuration from config file if it exists, otherwise use defaults."""
    config = DEFAULT_CONFIG.copy()

    # Check if config file exists
    if os.path.exists(config_path):
        try:
            with open(config_path, 'r') as f:
                user_config = json.load(f)
                config.update(user_config)
                print(f"Loaded configuration from {config_path}")
        except Exception as e:
            print(f"Error loading config file: {e}")
            print("Using default configuration")
    else:
        print(f"Config file not found at {config_path}. Using default configuration.")
        # Create a template config file for future use
        try:
            with open(config_path, 'w') as f:
                json.dump(DEFAULT_CONFIG, f, indent=4)
                print(f"Created template config file at {config_path}")
        except Exception as e:
            print(f"Failed to create template config file: {e}")

    return config

def load_user_text():
    """Load user text from file if it exists."""
    if os.path.exists(user_text_path):
        try:
            with open(user_text_path, 'r') as f:
                content = f.read().strip()
                print(f"Loaded user text from {user_text_path} ({len(content)} characters)")
                return content
        except Exception as e:
            print(f"Error loading user text file: {e}")
            return None
    else:
        print(f"User text file not found at {user_text_path}")
        return None

def update_config_interactive(config):
    """Allow user to interactively update configuration parameters."""
    print("\n===== Generation Configuration =====")
    print("Current settings:")
    for key, value in config.items():
        print(f"  {key}: {value}")

    print("\nEnter new values (press Enter to keep current value):")

    # Update numerical parameters
    for param in ["max_new_tokens", "temperature", "top_k"]:
        new_value = input(f"{param} [{config[param]}]: ")
        if new_value.strip():
            try:
                if param == "max_new_tokens" or param == "top_k":
                    config[param] = int(new_value)
                else:
                    config[param] = float(new_value)
            except ValueError:
                print(f"Invalid value for {param}, keeping current value.")

    # Update personalization parameters
    for param in ["consider_wardrobe", "consider_user_details"]:
        new_value = input(f"{param} [{config[param]}] (yes/no): ")
        if new_value.strip().lower() in ['yes', 'no']:
            config[param] = new_value.lower()

    # Save updated config
    try:
        with open(config_path, 'w') as f:
            json.dump(config, f, indent=4)
            print(f"Configuration saved to {config_path}")
    except Exception as e:
        print(f"Failed to save configuration: {e}")

    return config

def save_user_text(text):
    """Save user text to file."""
    try:
        with open(user_text_path, 'w') as f:
            f.write(text)
        print(f"Text saved to {user_text_path}")
    except Exception as e:
        print(f"Failed to save text: {e}")

def process_user_input_file(model, tokenizer, device, user_profile):
    """Process user input from file and generate response."""
    # Load configuration
    config = load_user_config()

    while True:
        # Get user text
        user_text = load_user_text()
        if user_text is None:
            print("Would you like to create a user.txt file? (y/n)")
            if input().lower() == 'y':
                print("Enter your text (end with a newline and Ctrl+D):")
                text = []
                try:
                    while True:
                        line = input()
                        text.append(line)
                except EOFError:
                    user_text = '\n'.join(text)
                    save_user_text(user_text)
            else:
                print("Switching to interactive mode...")
                return test_model_interactive(model, tokenizer, device, user_profile)

        # Parse instruction and input from user text
        parts = user_text.split('|', 1)
        config["instruction"] = parts[0].strip()
        config["input_text"] = parts[1].strip() if len(parts) > 1 else ""

        # Allow user to adjust generation parameters
        print("\nWould you like to adjust generation parameters? (y/n)")
        if input().lower() == 'y':
            config = update_config_interactive(config)

        # Format the prompt with personalization options
        prompt = format_input(
            config["instruction"],
            config["input_text"],
            config["consider_wardrobe"],
            config["consider_user_details"],
            user_profile
        )

        # Generate response
        print("\nGenerating response...")
        response, tokens = generate_text(
            model, tokenizer, prompt, device,
            max_new_tokens=config["max_new_tokens"],
            temperature=config["temperature"],
            top_k=config["top_k"]
        )

        # Display results
        print("\n=== Generated Response ===")
        print(response)
        print("\n=== Token Stats ===")
        print(f"Generated {len(tokens)} tokens")
        print(f"Input tokens: {len(tokenizer.encode(prompt))}")
        print(f"Total tokens: {len(tokenizer.encode(prompt)) + len(tokens)}")
        print(f"Personalization: Wardrobe = {config['consider_wardrobe']}, User Details = {config['consider_user_details']}")

        # Save response
        response_path = os.path.join(os.path.dirname(user_text_path), "response.txt")
        try:
            with open(response_path, 'w') as f:
                f.write(response)
            print(f"Response saved to {response_path}")
        except Exception as e:
            print(f"Failed to save response: {e}")

        # Ask to continue
        print("\nOptions:")
        print("1. Process the same text with different parameters")
        print("2. Edit the user.txt file")
        print("3. Switch to interactive mode")
        print("4. Exit")
        choice = input("Choose an option (1-4): ")

        if choice == '1':
            continue
        elif choice == '2':
            print("Enter new text (end with a newline and Ctrl+D):")
            text = []
            try:
                while True:
                    line = input()
                    text.append(line)
            except EOFError:
                user_text = '\n'.join(text)
                save_user_text(user_text)
        elif choice == '3':
            return test_model_interactive(model, tokenizer, device, user_profile)
        else:
            break

def test_model_interactive(model, tokenizer, device, user_profile):
    """Interactive testing of the model with custom prompts."""
    print("\n===== Interactive Testing Mode =====")
    print("Enter your queries to test the model. Type 'exit' to quit.")
    print("Format: instruction | input (optional)")

    # Provide helpful suggestions
    print("\nSuggested fashion-related queries:")
    print("1. Describe this outfit: | A black leather jacket with white t-shirt and blue jeans")
    print("2. Suggest accessories for | a summer wedding guest outfit with a floral dress")
    print("3. Compare these styles: | minimalist vs maximalist fashion")
    print("4. Create a shopping list for | building a capsule wardrobe for fall")
    print("5. What are current trends in | sustainable fashion for 2024?")

    # Load default config for generation parameters
    config = load_user_config()

    while True:
        # Show current personalization settings
        print(f"\nCurrent personalization: Consider Wardrobe = {config['consider_wardrobe']}, Consider User Details = {config['consider_user_details']}")

        # Ask to adjust generation parameters
        print("\nWould you like to adjust generation parameters? (y/n)")
        if input().lower() == 'y':
            config = update_config_interactive(config)

        user_input = input("\nYour query: ")

        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'file':
            return process_user_input_file(model, tokenizer, device, user_profile)

        # Parse instruction and optional input
        parts = user_input.split('|', 1)
        instruction = parts[0].strip()
        input_text = parts[1].strip() if len(parts) > 1 else ""

        # Format the prompt with personalization options
        prompt = format_input(
            instruction,
            input_text,
            config["consider_wardrobe"],
            config["consider_user_details"],
            user_profile
        )

        print("\nGenerating response...")
        response, tokens = generate_text(
            model, tokenizer, prompt, device,
            max_new_tokens=config["max_new_tokens"],
            temperature=config["temperature"],
            top_k=config["top_k"]
        )

        print("\n=== Generated Response ===")
        print(response)
        print("\n=== Token Stats ===")
        print(f"Generated {len(tokens)} tokens")
        print(f"Personalization: Wardrobe = {config['consider_wardrobe']}, User Details = {config['consider_user_details']}")

def update_user_profile(user_profile):
    """Interactive function to update user profile data"""
    print("\n===== Update User Profile =====")

    if user_profile.user_data:
        print("Current profile:")
        for key, value in user_profile.user_data.items():
            if key != 'wardrobe':  # Display wardrobe separately
                print(f"  {key}: {value}")

        # Display wardrobe items separately
        if user_profile.user_wardrobe:
            print("Wardrobe items:")
            for i, item in enumerate(user_profile.user_wardrobe, 1):
                print(f"  {i}. {item}")
    else:
        print("No profile data found. Creating new profile.")

    print("\nOptions:")
    print("1. Edit basic profile information")
    print("2. Edit wardrobe items")
    print("3. Cancel")

    choice = input("Choose an option (1-3): ")

    if choice == '1':
        # Edit basic profile info
        print("\nEnter profile information (press Enter to keep current values, 'delete' to remove):")
        fields = ['name', 'age', 'gender', 'location', 'style', 'occupation', 'preferences', 'favorite_colors', 'size']

        updated_data = {}
        for field in fields:
            current = user_profile.user_data.get(field, '')
            new_value = input(f"{field} [{current}]: ")

            if new_value.lower() == 'delete':
                # Don't include this field
                pass
            elif new_value.strip():
                # Update with new value
                updated_data[field] = new_value
            elif field in user_profile.user_data:
                # Keep current value
                updated_data[field] = current

        # Preserve wardrobe data
        if 'wardrobe' in user_profile.user_data:
            updated_data['wardrobe'] = user_profile.user_data['wardrobe']

        # Update user data
        user_profile.user_data = updated_data

        # Save to file
        save_updated_profile(user_profile)

    elif choice == '2':
        # Edit wardrobe items
        print("\nOptions for wardrobe:")
        print("1. Add items")
        print("2. Remove items")
        print("3. Replace all items")
        print("4. Cancel")

        wardrobe_choice = input("Choose an option (1-4): ")

        if wardrobe_choice == '1':
            # Add items
            new_items = input("Enter items to add (comma separated): ")
            items_to_add = [item.strip() for item in new_items.split(',') if item.strip()]

            if 'wardrobe' not in user_profile.user_data:
                user_profile.user_data['wardrobe'] = []
                user_profile.user_wardrobe = []

            user_profile.user_wardrobe.extend(items_to_add)
            user_profile.user_data['wardrobe'] = user_profile.user_wardrobe

            save_updated_profile(user_profile)

        elif wardrobe_choice == '2':
            # Remove items
            if not user_profile.user_wardrobe:
                print("No wardrobe items to remove.")
                return

            print("Current items:")
            for i, item in enumerate(user_profile.user_wardrobe, 1):
                print(f"  {i}. {item}")

            to_remove = input("Enter numbers of items to remove (comma separated): ")
            try:
                indices = [int(idx.strip()) for idx in to_remove.split(',') if idx.strip()]
                # Sort in reverse to avoid index shifting during removal
                indices.sort(reverse=True)

                for idx in indices:
                    if 1 <= idx <= len(user_profile.user_wardrobe):
                        del user_profile.user_wardrobe[idx-1]

                user_profile.user_data['wardrobe'] = user_profile.user_wardrobe
                save_updated_profile(user_profile)

            except ValueError:
                print("Invalid input. Please enter numbers separated by commas.")

        elif wardrobe_choice == '3':
            # Replace all
            new_wardrobe = input("Enter new wardrobe items (comma separated): ")
            new_items = [item.strip() for item in new_wardrobe.split(',') if item.strip()]

            user_profile.user_wardrobe = new_items
            user_profile.user_data['wardrobe'] = new_items

            save_updated_profile(user_profile)

def save_updated_profile(user_profile):
    """Save updated profile to user.txt file"""
    try:
        # Format profile data for saving
        lines = []

        # Add basic profile fields first
        for key, value in user_profile.user_data.items():
            if key != 'wardrobe':  # Handle wardrobe separately
                lines.append(f"{key}: {value}")

        # Add wardrobe at the end
        if user_profile.user_wardrobe:
            lines.append(f"wardrobe: {', '.join(user_profile.user_wardrobe)}")

        # Write to file
        with open(user_profile.user_text_path, 'w') as f:
            f.write('\n'.join(lines))

        print(f"Profile updated and saved to {user_profile.user_text_path}")
        # Reload the profile to ensure everything is up to date
        user_profile.load_user_data()
        return True

    except Exception as e:
        print(f"Error saving profile: {e}")
        return False

def main():
    # Unzip the model file
    print("Unzipping model file...")
    extracted_model_path = extract_with_progress(zip_filepath, unzip_destination)

    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Initialize user profile manager
    user_profile = UserProfileManager(user_text_path)

    # Check if user profile needs to be created or updated
    if not user_profile.user_data:
        print("No user profile found. Would you like to create one? (y/n)")
        if input().lower() == 'y':
            update_user_profile(user_profile)
    else:
        print("User profile loaded. Would you like to update it? (y/n)")
        if input().lower() == 'y':
            update_user_profile(user_profile)

    # Load the fine-tuned model
    model = load_finetuned_model(extracted_model_path, device)

    # Main menu
    while True:
        print("\n===== Fashion Recommendation System =====")
        print("1. Interactive mode")
        print("2. Process input from file")
        print("3. Update user profile")
        print("4. Exit")

        choice = input("Choose an option (1-4): ")

        if choice == '1':
            test_model_interactive(model, tokenizer, device, user_profile)
        elif choice == '2':
            process_user_ninput_file(model, tokenizer, device, user_profile)
        elif choice == '3':
            update_user_profile(user_profile)
        else:
            print("Exiting. Thank you!")
            break

if __name__ == "__main__":
    main()

Unzipping model file...


Extracting files: 100%|██████████| 1/1 [01:39<00:00, 99.66s/it]


Extraction complete to /content/extracted_model/
Using device: cpu
Loaded user profile from /content/drive/MyDrive/fashion_files/user.txt
User profile loaded. Would you like to update it? (y/n)
n
Loading model from /content/extracted_model/best_model.pt
Model loaded successfully! Validation loss: 1.2289036760727565

===== Fashion Recommendation System =====
1. Interactive mode
2. Process input from file
3. Update user profile
4. Exit
Choose an option (1-4): 3

===== Update User Profile =====
Current profile:
  name: sriram
  age: 21
  gender: male
  location: indi
  style: minimalist
  occupation: student
  preferences: minimalistic, stylish, classy
  favorite_colors: navy, beige, white, black
  size: m

Options:
1. Edit basic profile information
2. Edit wardrobe items
3. Cancel
Choose an option (1-3): 2

Options for wardrobe:
1. Add items
2. Remove items
3. Replace all items
4. Cancel
Choose an option (1-4): 1
Enter items to add (comma separated): white button-up shirt, navy trousers,

Generating tokens: 100%|██████████| 200/200 [12:31<00:00,  3.76s/it]



=== Generated Response ===

Okay, let's get you looking sharp and feeling comfortable while maintaining a minimalist aesthetic.  While your current wardrobe items are a bit too casual, I can definitely help you put together a sharp and minimalist look.

Since you like navy and beige, we can definitely work with that.  White is a fantastic base.  You can incorporate that with a navy blazer, which is perfect for a casual, summery vibe.  You can also roll up the sleeves for a relaxed touch if the weather's cool.  Make sure the blazer isn’t too long, ensuring it’s proportionally flattering.  Since you’re a minimalist man, leaving the top button undone will create a clean, streamlined silhouette, especially with the t-shirt.

For the shoes, black leather loafers or brogues would look sharp.  Avoid overly chunky sneakers.   Since you like polos, think classic white or black chinos.

=== Token Stats ===
Generated 200 tokens
Input tokens: 291
Total tokens: 491
Personalization: Wardrobe = yes,

Generating tokens: 100%|██████████| 200/200 [09:15<00:00,  2.78s/it]



=== Generated Response ===

Okay, so, here's an evening outing where you want to make a statement dressed down.  Given your minimalist aesthetic, minimalist color palette, and preferences, I've got some ideas.  Since you like navy and beige, and you're a student, we can definitely work with that.

For dinner, I recommend starting with the simple white button-up shirt.  It's the foundation.  Underneath, go for a navy slim-fit chino pant or a navy blazer.  This provides a streamlined, polished look.  Choose a slim fit, not too baggy, which will flatter your build.  Ensure the trousers are well-fitting, not too tight, for a relaxed vibe.  For footwear, black or dark brown leather loafers or brogues would work well.  Again, black or dark brown is versatile, and loafers elevate the look.  Finish with a simple white polo t-shirt and black denim jeans

=== Token Stats ===
Generated 200 tokens
Personalization: Wardrobe = yes, User Details = yes

Current personalization: Consider Wardrobe = ye

KeyboardInterrupt: Interrupted by user

# Inference based on user details

# VLM PART

In [ ]:
import zipfile

# Replace 'your_file.zip' with the name of the uploaded zip file
with zipfile.ZipFile('/content/drive/MyDrive/paligemma_model.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/Files')

In [ ]:
!pip install sympy==1.12.0 fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 111.2 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=1abcecc59248637a17979b13a8b2bbe0e6631b8e292eec16bc49840060560d2c
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.12 which is incompatible.


In [ ]:
# === modeling_siglip.py ===
from typing import Optional, Tuple
import torch
import torch.nn as nn

class SiglipVisionConfig:

    def __init__(
        self,
        hidden_size=768,
        intermediate_size=3072,
        num_hidden_layers=12,
        num_attention_heads=12,
        num_channels=3,
        image_size=224,
        patch_size=16,
        layer_norm_eps=1e-6,
        attention_dropout=0.0,
        num_image_tokens: int = None,
        **kwargs
    ):
        super().__init__()

        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.num_channels = num_channels
        self.patch_size = patch_size
        self.image_size = image_size
        self.attention_dropout = attention_dropout
        self.layer_norm_eps = layer_norm_eps
        self.num_image_tokens = num_image_tokens


class SiglipVisionEmbeddings(nn.Module):
    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        self.embed_dim = config.hidden_size
        self.image_size = config.image_size
        self.patch_size = config.patch_size

        self.patch_embedding = nn.Conv2d(
            in_channels=config.num_channels,
            out_channels=self.embed_dim,
            kernel_size=self.patch_size,
            stride=self.patch_size,
            padding="valid", # This indicates no padding is added
        )

        self.num_patches = (self.image_size // self.patch_size) ** 2
        self.num_positions = self.num_patches
        self.position_embedding = nn.Embedding(self.num_positions, self.embed_dim)
        self.register_buffer(
            "position_ids",
            torch.arange(self.num_positions).expand((1, -1)),
            persistent=False,
        )

    def forward(self, pixel_values: torch.FloatTensor) -> torch.Tensor:
        _, _, height, width = pixel_values.shape # [Batch_Size, Channels, Height, Width]
        # Convolve the `patch_size` kernel over the image, with no overlapping patches since the stride is equal to the kernel size
        # The output of the convolution will have shape [Batch_Size, Embed_Dim, Num_Patches_H, Num_Patches_W]
        # where Num_Patches_H = height // patch_size and Num_Patches_W = width // patch_size
        patch_embeds = self.patch_embedding(pixel_values)
        # [Batch_Size, Embed_Dim, Num_Patches_H, Num_Patches_W] -> [Batch_Size, Embed_Dim, Num_Patches]
        # where Num_Patches = Num_Patches_H * Num_Patches_W
        embeddings = patch_embeds.flatten(2)
        # [Batch_Size, Embed_Dim, Num_Patches] -> [Batch_Size, Num_Patches, Embed_Dim]
        embeddings = embeddings.transpose(1, 2)
        # Add position embeddings to each patch. Each positional encoding is a vector of size [Embed_Dim]
        embeddings = embeddings + self.position_embedding(self.position_ids)
        # [Batch_Size, Num_Patches, Embed_Dim]
        return embeddings


class SiglipAttention(nn.Module):
    """Multi-headed attention from 'Attention Is All You Need' paper"""

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.embed_dim = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = self.embed_dim // self.num_heads
        self.scale = self.head_dim**-0.5 # Equivalent to 1 / sqrt(self.head_dim)
        self.dropout = config.attention_dropout

        self.k_proj = nn.Linear(self.embed_dim, self.embed_dim)
        self.v_proj = nn.Linear(self.embed_dim, self.embed_dim)
        self.q_proj = nn.Linear(self.embed_dim, self.embed_dim)
        self.out_proj = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(
        self,
        hidden_states: torch.Tensor,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:

        # hidden_states: [Batch_Size, Num_Patches, Embed_Dim]
        batch_size, seq_len, _ = hidden_states.size()
        # query_states: [Batch_Size, Num_Patches, Embed_Dim]
        query_states = self.q_proj(hidden_states)
        # key_states: [Batch_Size, Num_Patches, Embed_Dim]
        key_states = self.k_proj(hidden_states)
        # value_states: [Batch_Size, Num_Patches, Embed_Dim]
        value_states = self.v_proj(hidden_states)
        # query_states: [Batch_Size, Num_Heads, Num_Patches, Head_Dim]
        query_states = query_states.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        key_states = key_states.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        value_states = value_states.view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        # Calculate the attention using the formula Q * K^T / sqrt(d_k). attn_weights: [Batch_Size, Num_Heads, Num_Patches, Num_Patches]
        attn_weights = (torch.matmul(query_states, key_states.transpose(2, 3)) * self.scale)

        if attn_weights.size() != (batch_size, self.num_heads, seq_len, seq_len):
            raise ValueError(
                f"Attention weights should be of size {(batch_size, self.num_heads, seq_len, seq_len)}, but is"
                f" {attn_weights.size()}"
            )

        # Apply the softmax row-wise. attn_weights: [Batch_Size, Num_Heads, Num_Patches, Num_Patches]
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        # Apply dropout only during training
        attn_weights = nn.functional.dropout(attn_weights, p=self.dropout, training=self.training)
        # Multiply the attention weights by the value states. attn_output: [Batch_Size, Num_Heads, Num_Patches, Head_Dim]
        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (batch_size, self.num_heads, seq_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(batch_size, self.num_heads, seq_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        # [Batch_Size, Num_Heads, Num_Patches, Head_Dim] -> [Batch_Size, Num_Patches, Num_Heads, Head_Dim]
        attn_output = attn_output.transpose(1, 2).contiguous()
        # [Batch_Size, Num_Patches, Num_Heads, Head_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
        attn_output = attn_output.reshape(batch_size, seq_len, self.embed_dim)
        # [Batch_Size, Num_Patches, Embed_Dim]
        attn_output = self.out_proj(attn_output)

        return attn_output, attn_weights


class SiglipMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.fc1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.fc2 = nn.Linear(config.intermediate_size, config.hidden_size)

    def forward(self, hidden_states: torch.Tensor) -> torch.Tensor:
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Intermediate_Size]
        hidden_states = self.fc1(hidden_states)
        # hidden_states: [Batch_Size, Num_Patches, Intermediate_Size]
        hidden_states = nn.functional.gelu(hidden_states, approximate="tanh")
        # [Batch_Size, Num_Patches, Intermediate_Size] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = self.fc2(hidden_states)

        return hidden_states


class SiglipEncoderLayer(nn.Module):
    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.embed_dim = config.hidden_size
        self.self_attn = SiglipAttention(config)
        self.layer_norm1 = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_eps)
        self.mlp = SiglipMLP(config)
        self.layer_norm2 = nn.LayerNorm(self.embed_dim, eps=config.layer_norm_eps)

    # Ignore copy
    def forward(
        self,
        hidden_states: torch.Tensor
    ) -> torch.Tensor:
        # residual: [Batch_Size, Num_Patches, Embed_Dim]
        residual = hidden_states
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = self.layer_norm1(hidden_states)
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states, _ = self.self_attn(hidden_states=hidden_states)
        # [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = residual + hidden_states
        # residual: [Batch_Size, Num_Patches, Embed_Dim]
        residual = hidden_states
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = self.layer_norm2(hidden_states)
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = self.mlp(hidden_states)
        # [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = residual + hidden_states

        return hidden_states


class SiglipEncoder(nn.Module):
    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        self.layers = nn.ModuleList(
            [SiglipEncoderLayer(config) for _ in range(config.num_hidden_layers)]
        )

    # Ignore copy
    def forward(
        self,
        inputs_embeds: torch.Tensor
    ) -> torch.Tensor:
        # inputs_embeds: [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = inputs_embeds

        for encoder_layer in self.layers:
            # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Embed_Dim]
            hidden_states = encoder_layer(hidden_states)

        return hidden_states


class SiglipVisionTransformer(nn.Module):
    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        embed_dim = config.hidden_size

        self.embeddings = SiglipVisionEmbeddings(config)
        self.encoder = SiglipEncoder(config)
        self.post_layernorm = nn.LayerNorm(embed_dim, eps=config.layer_norm_eps)

    def forward(self, pixel_values: torch.Tensor) -> torch.Tensor:
        # pixel_values: [Batch_Size, Channels, Height, Width] -> [Batch_Size, Num_Patches, Embed_Dim]
        hidden_states = self.embeddings(pixel_values)

        last_hidden_state = self.encoder(inputs_embeds=hidden_states)

        last_hidden_state = self.post_layernorm(last_hidden_state)

        return last_hidden_state


class SiglipVisionModel(nn.Module):

    def __init__(self, config: SiglipVisionConfig):
        super().__init__()
        self.config = config
        self.vision_model = SiglipVisionTransformer(config)

    def forward(self, pixel_values) -> Tuple:
        # [Batch_Size, Channels, Height, Width] -> [Batch_Size, Num_Patches, Embed_Dim]
        return self.vision_model(pixel_values=pixel_values)


# === modeling_gemma.py ===
import torch
from torch import nn
from typing import Optional, Tuple, List
from torch.nn import CrossEntropyLoss
import math


class KVCache():

    def __init__(self) -> None:
        self.key_cache: List[torch.Tensor] = []
        self.value_cache: List[torch.Tensor] = []

    def num_items(self) -> int:
        if len(self.key_cache) == 0:
            return 0
        else:
            # The shape of the key_cache is [Batch_Size, Num_Heads_KV, Seq_Len, Head_Dim]
            return self.key_cache[0].shape[-2]

    def update(
        self,
        key_states: torch.Tensor,
        value_states: torch.Tensor,
        layer_idx: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        if len(self.key_cache) <= layer_idx:
            # If we never added anything to the KV-Cache of this layer, let's create it.
            self.key_cache.append(key_states)
            self.value_cache.append(value_states)
        else:
            # ... otherwise we concatenate the new keys with the existing ones.
            # each tensor has shape: [Batch_Size, Num_Heads_KV, Seq_Len, Head_Dim]
            self.key_cache[layer_idx] = torch.cat([self.key_cache[layer_idx], key_states], dim=-2)
            self.value_cache[layer_idx] = torch.cat([self.value_cache[layer_idx], value_states], dim=-2)

        # ... and then we return all the existing keys + the new ones.
        return self.key_cache[layer_idx], self.value_cache[layer_idx]

class GemmaConfig():

    def __init__(
        self,
        vocab_size,
        hidden_size,
        intermediate_size,
        num_hidden_layers,
        num_attention_heads,
        num_key_value_heads,
        head_dim=256,
        max_position_embeddings=8192,
        rms_norm_eps=1e-6,
        rope_theta=10000.0,
        attention_bias=False,
        attention_dropout=0.0,
        pad_token_id=None,
        **kwargs,
    ):
        super().__init__()
        self.vocab_size = vocab_size
        self.max_position_embeddings = max_position_embeddings
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads
        self.head_dim = head_dim
        self.num_key_value_heads = num_key_value_heads
        self.rms_norm_eps = rms_norm_eps
        self.rope_theta = rope_theta
        self.attention_bias = attention_bias
        self.attention_dropout = attention_dropout
        self.pad_token_id = pad_token_id

class PaliGemmaConfig():

    def __init__(
        self,
        vision_config=None,
        text_config=None,
        ignore_index=-100,
        image_token_index=256000,
        vocab_size=257152,
        projection_dim=2048,
        hidden_size=2048,
        pad_token_id=None,
        **kwargs,
    ):
        super().__init__()
        self.ignore_index = ignore_index
        self.image_token_index = image_token_index
        self.vocab_size = vocab_size
        self.projection_dim = projection_dim
        self.hidden_size = hidden_size
        self.vision_config = vision_config
        self.is_encoder_decoder = False
        self.pad_token_id = pad_token_id

        self.vision_config = SiglipVisionConfig(**vision_config)
        self.text_config = text_config

        self.text_config = GemmaConfig(**text_config, pad_token_id=pad_token_id)
        self.vocab_size = self.text_config.vocab_size

        self.text_config.num_image_tokens = (self.vision_config.image_size // self.vision_config.patch_size) ** 2
        self.vision_config.projection_dim = projection_dim


class GemmaRMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-6):
        super().__init__()
        self.eps = eps
        self.weight = nn.Parameter(torch.zeros(dim))

    def _norm(self, x):
        return x * torch.rsqrt(x.pow(2).mean(-1, keepdim=True) + self.eps)

    def forward(self, x):
        output = self._norm(x.float())
        # Llama does x.to(float16) * w whilst Gemma is (x * w).to(float16)
        # See https://github.com/huggingface/transformers/pull/29402
        output = output * (1.0 + self.weight.float())
        return output.type_as(x)

class GemmaRotaryEmbedding(nn.Module):
    def __init__(self, dim, max_position_embeddings=2048, base=10000, device=None):
        super().__init__()

        self.dim = dim # it is set to the head_dim
        self.max_position_embeddings = max_position_embeddings
        self.base = base

        # Calculate the theta according to the formula theta_i = base^(-2i/dim) where i = 0, 1, 2, ..., dim // 2
        inv_freq = 1.0 / (self.base ** (torch.arange(0, self.dim, 2, dtype=torch.int64).float() / self.dim))
        self.register_buffer("inv_freq", tensor=inv_freq, persistent=False)

    @torch.no_grad()
    def forward(self, x, position_ids, seq_len=None):
        # x: [bs, num_attention_heads, seq_len, head_size]
        self.inv_freq.to(x.device)
        # Copy the inv_freq tensor for batch in the sequence
        # inv_freq_expanded: [Batch_Size, Head_Dim // 2, 1]
        inv_freq_expanded = self.inv_freq[None, :, None].float().expand(position_ids.shape[0], -1, 1)
        # position_ids_expanded: [Batch_Size, 1, Seq_Len]
        position_ids_expanded = position_ids[:, None, :].float()
        device_type = x.device.type
        device_type = device_type if isinstance(device_type, str) and device_type != "mps" else "cpu"
        with torch.autocast(device_type=device_type, enabled=False):
            # Multiply each theta by the position (which is the argument of the sin and cos functions)
            # freqs: [Batch_Size, Head_Dim // 2, 1] @ [Batch_Size, 1, Seq_Len] --> [Batch_Size, Seq_Len, Head_Dim // 2]
            freqs = (inv_freq_expanded.float() @ position_ids_expanded.float()).transpose(1, 2)
            # emb: [Batch_Size, Seq_Len, Head_Dim]
            emb = torch.cat((freqs, freqs), dim=-1)
            # cos, sin: [Batch_Size, Seq_Len, Head_Dim]
            cos = emb.cos()
            sin = emb.sin()
        return cos.to(dtype=x.dtype), sin.to(dtype=x.dtype)


def rotate_half(x):
    # Build the [-x2, x1, -x4, x3, ...] tensor for the sin part of the positional encoding.
    x1 = x[..., : x.shape[-1] // 2] # Takes the first half of the last dimension
    x2 = x[..., x.shape[-1] // 2 :] # Takes the second half of the last dimension
    return torch.cat((-x2, x1), dim=-1)


def apply_rotary_pos_emb(q, k, cos, sin, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim) # Add the head dimension
    sin = sin.unsqueeze(unsqueeze_dim) # Add the head dimension
    # Apply the formula (34) of the Rotary Positional Encoding paper.
    q_embed = (q * cos) + (rotate_half(q) * sin)
    k_embed = (k * cos) + (rotate_half(k) * sin)
    return q_embed, k_embed


class GemmaMLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.hidden_size = config.hidden_size
        self.intermediate_size = config.intermediate_size
        self.gate_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.up_proj = nn.Linear(self.hidden_size, self.intermediate_size, bias=False)
        self.down_proj = nn.Linear(self.intermediate_size, self.hidden_size, bias=False)

    def forward(self, x):
        # Equivalent to:
        # y = self.gate_proj(x) # [Batch_Size, Seq_Len, Hidden_Size] -> [Batch_Size, Seq_Len, Intermediate_Size]
        # y = torch.gelu(y, approximate="tanh") # [Batch_Size, Seq_Len, Intermediate_Size]
        # j = self.up_proj(x) # [Batch_Size, Seq_Len, Hidden_Size] -> [Batch_Size, Seq_Len, Intermediate_Size]
        # z = y * j # [Batch_Size, Seq_Len, Intermediate_Size]
        # z = self.down_proj(z) # [Batch_Size, Seq_Len, Intermediate_Size] -> [Batch_Size, Seq_Len, Hidden_Size]
        return self.down_proj(nn.functional.gelu(self.gate_proj(x), approximate="tanh") * self.up_proj(x))

def repeat_kv(hidden_states: torch.Tensor, n_rep: int) -> torch.Tensor:
    batch, num_key_value_heads, slen, head_dim = hidden_states.shape
    if n_rep == 1:
        return hidden_states
    hidden_states = hidden_states[:, :, None, :, :].expand(batch, num_key_value_heads, n_rep, slen, head_dim)
    return hidden_states.reshape(batch, num_key_value_heads * n_rep, slen, head_dim)

class GemmaAttention(nn.Module):

    def __init__(self, config: GemmaConfig, layer_idx: Optional[int] = None):
        super().__init__()
        self.config = config
        self.layer_idx = layer_idx

        self.attention_dropout = config.attention_dropout
        self.hidden_size = config.hidden_size
        self.num_heads = config.num_attention_heads
        self.head_dim = config.head_dim
        self.num_key_value_heads = config.num_key_value_heads
        self.num_key_value_groups = self.num_heads // self.num_key_value_heads
        self.max_position_embeddings = config.max_position_embeddings
        self.rope_theta = config.rope_theta
        self.is_causal = True

        assert self.hidden_size % self.num_heads == 0

        self.q_proj = nn.Linear(self.hidden_size, self.num_heads * self.head_dim, bias=config.attention_bias)
        self.k_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=config.attention_bias)
        self.v_proj = nn.Linear(self.hidden_size, self.num_key_value_heads * self.head_dim, bias=config.attention_bias)
        self.o_proj = nn.Linear(self.num_heads * self.head_dim, self.hidden_size, bias=config.attention_bias)
        self.rotary_emb = GemmaRotaryEmbedding(
            self.head_dim,
            max_position_embeddings=self.max_position_embeddings,
            base=self.rope_theta,
        )

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        kv_cache: Optional[KVCache] = None,
        **kwargs,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
        bsz, q_len, _ = hidden_states.size() # [Batch_Size, Seq_Len, Hidden_Size]
        # [Batch_Size, Seq_Len, Num_Heads_Q * Head_Dim]
        query_states = self.q_proj(hidden_states)
        # [Batch_Size, Seq_Len, Num_Heads_KV * Head_Dim]
        key_states = self.k_proj(hidden_states)
        # [Batch_Size, Seq_Len, Num_Heads_KV * Head_Dim]
        value_states = self.v_proj(hidden_states)
        # [Batch_Size, Num_Heads_Q, Seq_Len, Head_Dim]
        query_states = query_states.view(bsz, q_len, self.num_heads, self.head_dim).transpose(1, 2)
        # [Batch_Size, Num_Heads_KV, Seq_Len, Head_Dim]
        key_states = key_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)
        # [Batch_Size, Num_Heads_KV, Seq_Len, Head_Dim]
        value_states = value_states.view(bsz, q_len, self.num_key_value_heads, self.head_dim).transpose(1, 2)

        # [Batch_Size, Seq_Len, Head_Dim], [Batch_Size, Seq_Len, Head_Dim]
        cos, sin = self.rotary_emb(value_states, position_ids, seq_len=None)
        # [Batch_Size, Num_Heads_Q, Seq_Len, Head_Dim], [Batch_Size, Num_Heads_KV, Seq_Len, Head_Dim]
        query_states, key_states = apply_rotary_pos_emb(query_states, key_states, cos, sin)

        if kv_cache is not None:
            key_states, value_states = kv_cache.update(key_states, value_states, self.layer_idx)

        # Repeat the key and values to match the number of heads of the query
        key_states = repeat_kv(key_states, self.num_key_value_groups)
        value_states = repeat_kv(value_states, self.num_key_value_groups)
        # Perform the calculation as usual, Q * K^T / sqrt(head_dim). Shape: [Batch_Size, Num_Heads_Q, Seq_Len_Q, Seq_Len_KV]
        attn_weights = torch.matmul(query_states, key_states.transpose(2, 3)) / math.sqrt(self.head_dim)

        assert attention_mask is not None
        attn_weights = attn_weights + attention_mask

        # Apply the softmax
        # [Batch_Size, Num_Heads_Q, Seq_Len_Q, Seq_Len_KV]
        attn_weights = nn.functional.softmax(attn_weights, dim=-1, dtype=torch.float32).to(query_states.dtype)
        # Apply the dropout
        attn_weights = nn.functional.dropout(attn_weights, p=self.attention_dropout, training=self.training)
        # Multiply by the values. [Batch_Size, Num_Heads_Q, Seq_Len_Q, Seq_Len_KV] x [Batch_Size, Num_Heads_KV, Seq_Len_KV, Head_Dim] -> [Batch_Size, Num_Heads_Q, Seq_Len_Q, Head_Dim]
        attn_output = torch.matmul(attn_weights, value_states)

        if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
            raise ValueError(
                f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
                f" {attn_output.size()}"
            )
        # Make sure the sequence length is the second dimension. # [Batch_Size, Num_Heads_Q, Seq_Len_Q, Head_Dim] -> [Batch_Size, Seq_Len_Q, Num_Heads_Q, Head_Dim]
        attn_output = attn_output.transpose(1, 2).contiguous()
        # Concatenate all the heads together. [Batch_Size, Seq_Len_Q, Num_Heads_Q, Head_Dim] -> [Batch_Size, Seq_Len_Q, Num_Heads_Q * Head_Dim]
        attn_output = attn_output.view(bsz, q_len, -1)
        # Multiply by W_o. [Batch_Size, Seq_Len_Q, Hidden_Size]
        attn_output = self.o_proj(attn_output)

        return attn_output, attn_weights

class GemmaDecoderLayer(nn.Module):

    def __init__(self, config: GemmaConfig, layer_idx: int):
        super().__init__()
        self.hidden_size = config.hidden_size

        self.self_attn = GemmaAttention(config=config, layer_idx=layer_idx)

        self.mlp = GemmaMLP(config)
        self.input_layernorm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)
        self.post_attention_layernorm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def forward(
        self,
        hidden_states: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        kv_cache: Optional[KVCache] = None,
    ) -> Tuple[torch.FloatTensor, Optional[Tuple[torch.FloatTensor, torch.FloatTensor]]]:
        residual = hidden_states
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = self.input_layernorm(hidden_states)

        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states, _, = self.self_attn(
            hidden_states=hidden_states,
            attention_mask=attention_mask,
            position_ids=position_ids,
            kv_cache=kv_cache,
        )
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = residual + hidden_states

        # [Batch_Size, Seq_Len, Hidden_Size]
        residual = hidden_states
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = self.post_attention_layernorm(hidden_states)
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = self.mlp(hidden_states)
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = residual + hidden_states

        return hidden_states

class GemmaModel(nn.Module):

    def __init__(self, config: GemmaConfig):
        super().__init__()
        self.config = config
        self.padding_idx = config.pad_token_id
        self.vocab_size = config.vocab_size

        self.embed_tokens = nn.Embedding(config.vocab_size, config.hidden_size, self.padding_idx)
        self.layers = nn.ModuleList(
            [GemmaDecoderLayer(config, layer_idx) for layer_idx in range(config.num_hidden_layers)]
        )
        self.norm = GemmaRMSNorm(config.hidden_size, eps=config.rms_norm_eps)

    def get_input_embeddings(self):
        return self.embed_tokens

    # Ignore copy
    def forward(
        self,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        kv_cache: Optional[KVCache] = None,
    ) -> torch.FloatTensor:
        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = inputs_embeds
        # [Batch_Size, Seq_Len, Hidden_Size]
        normalizer = torch.tensor(self.config.hidden_size**0.5, dtype=hidden_states.dtype)
        hidden_states = hidden_states * normalizer

        for decoder_layer in self.layers:
            # [Batch_Size, Seq_Len, Hidden_Size]
            hidden_states = decoder_layer(
                hidden_states,
                attention_mask=attention_mask,
                position_ids=position_ids,
                kv_cache=kv_cache,
            )

        # [Batch_Size, Seq_Len, Hidden_Size]
        hidden_states = self.norm(hidden_states)

        # [Batch_Size, Seq_Len, Hidden_Size]
        return hidden_states

class GemmaForCausalLM(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = config
        self.model = GemmaModel(config)
        self.vocab_size = config.vocab_size
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)

    def get_input_embeddings(self):
        return self.model.embed_tokens

    def tie_weights(self):
        self.lm_head.weight = self.model.embed_tokens.weight

    def forward(
        self,
        attention_mask: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.LongTensor] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        kv_cache: Optional[KVCache] = None,
    ) -> Tuple:

        # input_embeds: [Batch_Size, Seq_Len, Hidden_Size]
        # outputs: [Batch_Size, Seq_Len, Hidden_Size]
        outputs = self.model(
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            kv_cache=kv_cache,
        )

        hidden_states = outputs
        logits = self.lm_head(hidden_states)
        logits = logits.float()

        return_data = {
            "logits": logits,
        }

        if kv_cache is not None:
            # Return the updated cache
            return_data["kv_cache"] = kv_cache

        return return_data

class PaliGemmaMultiModalProjector(nn.Module):
    def __init__(self, config: PaliGemmaConfig):
        super().__init__()
        self.linear = nn.Linear(config.vision_config.hidden_size, config.vision_config.projection_dim, bias=True)

    def forward(self, image_features):
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Projection_Dim]
        hidden_states = self.linear(image_features)
        return hidden_states

class PaliGemmaForConditionalGeneration(nn.Module):
    def __init__(self, config: PaliGemmaConfig):
        super().__init__()
        self.config = config
        self.vision_tower = SiglipVisionModel(config.vision_config)
        self.multi_modal_projector = PaliGemmaMultiModalProjector(config)
        self.vocab_size = config.vocab_size

        language_model = GemmaForCausalLM(config.text_config)
        self.language_model = language_model

        self.pad_token_id = self.config.pad_token_id if self.config.pad_token_id is not None else -1

    def tie_weights(self):
        return self.language_model.tie_weights()

    def _merge_input_ids_with_image_features(self, image_features: torch.Tensor, inputs_embeds: torch.Tensor, input_ids: torch.Tensor, attention_mask: torch.Tensor, kv_cache: Optional[KVCache] = None):
        batch_size, num_patches, embed_dim = image_features.shape
        batch_size_input, sequence_length = input_ids.shape
        dtype, device = inputs_embeds.dtype, inputs_embeds.device

        # Shape: [Batch_Size, Seq_Len, Hidden_Size]
        scaled_image_features = image_features / (self.config.hidden_size**0.5)

        # Combine the embeddings of the image tokens, the text tokens and mask out all the padding tokens.
        final_embedding = torch.zeros(batch_size, sequence_length, embed_dim, dtype=inputs_embeds.dtype, device=inputs_embeds.device)

        # Shape: [Batch_Size, Seq_Len]. True for text tokens
        text_mask = (input_ids != self.config.image_token_index) & (input_ids != self.pad_token_id)

        # Shape: [Batch_Size, Seq_Len]. True for image tokens
        image_mask = input_ids == self.config.image_token_index

        # Shape: [Batch_Size, Seq_Len]. True for padding tokens
        pad_mask = input_ids == self.pad_token_id

        # We need to expand the masks to the embedding dimension otherwise we can't use them in torch.where
        text_mask_expanded = text_mask.unsqueeze(-1).expand(-1, -1, embed_dim)
        pad_mask_expanded = pad_mask.unsqueeze(-1).expand(-1, -1, embed_dim)
        image_mask_expanded = image_mask.unsqueeze(-1).expand(-1, -1, embed_dim)

        # Add the text embeddings
        final_embedding = torch.where(text_mask_expanded, inputs_embeds, final_embedding)

        # FIX: Count the number of image tokens per batch item and ensure we have enough features
        for batch_idx in range(batch_size):
            image_token_positions = torch.where(image_mask[batch_idx])[0]
            num_image_tokens = len(image_token_positions)

            # Make sure we don't try to insert more features than we have
            if num_image_tokens > 0:
                # If we have more image tokens than features, we need to repeat the features
                # or use a subset of the tokens depending on requirements
                features_to_use = scaled_image_features[batch_idx, :num_image_tokens]

                # Insert image embeddings for each image token position
                for token_idx, pos in enumerate(image_token_positions):
                    if token_idx < len(features_to_use):
                        final_embedding[batch_idx, pos] = features_to_use[token_idx]

        # Zero out padding tokens
        final_embedding = torch.where(pad_mask_expanded, torch.zeros_like(final_embedding), final_embedding)

        #### CREATE THE ATTENTION MASK ####

        dtype, device = inputs_embeds.dtype, inputs_embeds.device
        min_dtype = torch.finfo(dtype).min
        q_len = inputs_embeds.shape[1]

        if kv_cache is None or kv_cache.num_items() == 0:
            # Do not mask any token, because we're in the prefill phase
            # This only works when we have no padding
            causal_mask = torch.full(
                (batch_size, q_len, q_len), fill_value=0, dtype=dtype, device=device
            )
        else:
            # Since we are generating tokens, the query must be one single token
            assert q_len == 1
            kv_len = kv_cache.num_items() + q_len
            # Also in this case we don't need to mask anything, since each query should be able to attend all previous tokens.
            # This only works when we have no padding
            causal_mask = torch.full(
                (batch_size, q_len, kv_len), fill_value=0, dtype=dtype, device=device
            )

        # Add the head dimension
        # [Batch_Size, Q_Len, KV_Len] -> [Batch_Size, Num_Heads_Q, Q_Len, KV_Len]
        causal_mask = causal_mask.unsqueeze(1)

        if kv_cache is not None and kv_cache.num_items() > 0:
            # The position of the query is just the last position
            position_ids = attention_mask.cumsum(-1)[:, -1]
            if position_ids.dim() == 1:
                position_ids = position_ids.unsqueeze(0)
        else:
            # Create a position_ids based on the size of the attention_mask
            # For masked tokens, use the number 1 as position.
            position_ids = (attention_mask.cumsum(-1)).masked_fill_((attention_mask == 0), 1).to(device)

        return final_embedding, causal_mask, position_ids

    def forward(
        self,
        input_ids: torch.LongTensor = None,
        pixel_values: torch.FloatTensor = None,
        attention_mask: Optional[torch.Tensor] = None,
        kv_cache: Optional[KVCache] = None,
    ) -> Tuple:

        # Make sure the input is right-padded
        #assert torch.all(attention_mask == 1), "The input cannot be padded"

        # 1. Extra the input embeddings
        # shape: (Batch_Size, Seq_Len, Hidden_Size)
        inputs_embeds = self.language_model.get_input_embeddings()(input_ids)

        # 2. Merge text and images
        # [Batch_Size, Channels, Height, Width] -> [Batch_Size, Num_Patches, Embed_Dim]
        selected_image_feature = self.vision_tower(pixel_values.to(inputs_embeds.dtype))
        # [Batch_Size, Num_Patches, Embed_Dim] -> [Batch_Size, Num_Patches, Hidden_Size]
        image_features = self.multi_modal_projector(selected_image_feature)

        # Merge the embeddings of the text tokens and the image tokens
        inputs_embeds, attention_mask, position_ids = self._merge_input_ids_with_image_features(image_features, inputs_embeds, input_ids, attention_mask, kv_cache)

        outputs = self.language_model(
            attention_mask=attention_mask,
            position_ids=position_ids,
            inputs_embeds=inputs_embeds,
            kv_cache=kv_cache,
        )

        return outputs


# === utils.py ===
from transformers import AutoTokenizer
import json
import glob
from safetensors import safe_open
from typing import Tuple
import os

def load_hf_model(model_path: str, device: str) -> Tuple[PaliGemmaForConditionalGeneration, AutoTokenizer]:
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
    assert tokenizer.padding_side == "right"

    # Find all the *.safetensors files
    safetensors_files = glob.glob(os.path.join(model_path, "*.safetensors"))

    # ... and load them one by one in the tensors dictionary
    tensors = {}
    for safetensors_file in safetensors_files:
        with safe_open(safetensors_file, framework="pt", device="cpu") as f:
            for key in f.keys():
                tensors[key] = f.get_tensor(key)

    # Load the model's config
    with open(os.path.join(model_path, "config.json"), "r") as f:
        model_config_file = json.load(f)
        config = PaliGemmaConfig(**model_config_file)

    # Create the model using the configuration
    model = PaliGemmaForConditionalGeneration(config).to(device)

    # Load the state dict of the model
    model.load_state_dict(tensors, strict=False)

    # Tie weights
    model.tie_weights()

    return (model, tokenizer)

# === processing_paligemma.py ===
from typing import Dict, List, Optional, Union, Tuple, Iterable
import numpy as np
from PIL import Image
import torch

IMAGENET_STANDARD_MEAN = [0.5, 0.5, 0.5]
IMAGENET_STANDARD_STD = [0.5, 0.5, 0.5]


def add_image_tokens_to_prompt(prefix_prompt, bos_token, image_seq_len, image_token):
    # Quoting from the blog (https://huggingface.co/blog/paligemma#detailed-inference-process):
    #   The input text is tokenized normally.
    #   A <bos> token is added at the beginning, and an additional newline token (\n) is appended.
    #   This newline token is an essential part of the input prompt the model was trained with, so adding it explicitly ensures it's always there.
    #   The tokenized text is also prefixed with a fixed number of <image> tokens.
    # NOTE: from the paper it looks like the `\n` should be tokenized separately, but in the HF implementation this is not done.
    #       ref to HF implementation: https://github.com/huggingface/transformers/blob/7f79a97399bb52aad8460e1da2f36577d5dccfed/src/transformers/models/paligemma/processing_paligemma.py#L55-L73
    return f"{image_token * image_seq_len}{bos_token}{prefix_prompt}\n"


def rescale(
    image: np.ndarray, scale: float, dtype: np.dtype = np.float32
) -> np.ndarray:
    rescaled_image = image * scale
    rescaled_image = rescaled_image.astype(dtype)
    return rescaled_image


def resize(
    image: Image,
    size: Tuple[int, int],
    resample: Image.Resampling = None,
    reducing_gap: Optional[int] = None,
) -> np.ndarray:
    height, width = size
    resized_image = image.resize(
        (width, height), resample=resample, reducing_gap=reducing_gap
    )
    return resized_image


def normalize(
    image: np.ndarray,
    mean: Union[float, Iterable[float]],
    std: Union[float, Iterable[float]],
) -> np.ndarray:
    mean = np.array(mean, dtype=image.dtype)
    std = np.array(std, dtype=image.dtype)
    image = (image - mean) / std
    return image


def process_images(
    images: List[Image.Image],
    size: Dict[str, int] = None,
    resample: Image.Resampling = None,
    rescale_factor: float = None,
    image_mean: Optional[Union[float, List[float]]] = None,
    image_std: Optional[Union[float, List[float]]] = None,
) -> List[np.ndarray]:
    height, width = size[0], size[1]
    images = [
        resize(image=image, size=(height, width), resample=resample) for image in images
    ]
    # Convert each image to a numpy array
    images = [np.array(image) for image in images]
    # Rescale the pixel values to be in the range [0, 1]
    images = [rescale(image, scale=rescale_factor) for image in images]
    # Normalize the images to have mean 0 and standard deviation 1
    images = [normalize(image, mean=image_mean, std=image_std) for image in images]
    # Move the channel dimension to the first dimension. The model expects images in the format [Channel, Height, Width]
    images = [image.transpose(2, 0, 1) for image in images]
    return images


class PaliGemmaProcessor:
    IMAGE_TOKEN = "<image>"

    def __init__(self, tokenizer, num_image_tokens: int, image_size: int):
        super().__init__()

        self.image_seq_length = num_image_tokens
        self.image_size = image_size

        # Tokenizer described here: https://github.com/google-research/big_vision/blob/main/big_vision/configs/proj/paligemma/README.md#tokenizer
        tokens_to_add = {"additional_special_tokens": [self.IMAGE_TOKEN]}
        tokenizer.add_special_tokens(tokens_to_add)
        EXTRA_TOKENS = [
            f"<loc{i:04d}>" for i in range(1024)
        ]  # These tokens are used for object detection (bounding boxes)
        EXTRA_TOKENS += [
            f"<seg{i:03d}>" for i in range(128)
        ]  # These tokens are used for object segmentation
        tokenizer.add_tokens(EXTRA_TOKENS)
        self.image_token_id = tokenizer.convert_tokens_to_ids(self.IMAGE_TOKEN)
        # We will add the BOS and EOS tokens ourselves
        tokenizer.add_bos_token = False
        tokenizer.add_eos_token = False

        self.tokenizer = tokenizer

    def __call__(
        self,
        text: List[str],
        images: List[Image.Image],
        suffix: Optional[List[str]] = None,
        padding: str = "longest",
        truncation: bool = True,
        return_tensors: str = "pt",
    ) -> dict:
        assert len(images) == 1 and len(text) == 1, f"Received {len(images)} images for {len(text)} prompts."

        # Process images
        pixel_values = process_images(
            images,
            size=(self.image_size, self.image_size),
            resample=Image.Resampling.BICUBIC,
            rescale_factor=1 / 255.0,
            image_mean=IMAGENET_STANDARD_MEAN,
            image_std=IMAGENET_STANDARD_STD,
        )
        # Convert the list of numpy arrays to a single numpy array with shape [Batch_Size, Channel, Height, Width]
        pixel_values = np.stack(pixel_values, axis=0)
        # Convert the numpy array to a PyTorch tensor
        pixel_values = torch.tensor(pixel_values)

        # Prepend a `self.image_seq_length` number of image tokens to the prompt
        input_strings = [
            add_image_tokens_to_prompt(
                prefix_prompt=prompt,
                bos_token=self.tokenizer.bos_token,
                image_seq_len=self.image_seq_length,
                image_token=self.IMAGE_TOKEN,
            )
            for prompt in text
        ]

        # Process inputs based on whether suffix is provided
        if suffix is not None:
            assert len(suffix) == len(text), "Number of suffixes must match number of prompts"

            # Tokenize the input prompts
            inputs = self.tokenizer(
                input_strings,
                return_tensors=return_tensors,
                padding=padding,
                truncation=truncation,
            )

            # Tokenize the suffixes
            suffix_encodings = self.tokenizer(
                suffix,
                return_tensors=return_tensors,
                padding=padding,
                truncation=truncation,
            )

            # Create token_type_ids: 0 for prompt, 1 for suffix
            batch_size, seq_len = inputs["input_ids"].shape
            token_type_ids = torch.zeros_like(inputs["input_ids"])

            # Create labels for training
            # Start with -100 (ignored in loss calculation)
            labels = torch.full_like(inputs["input_ids"], fill_value=-100)

            # Concatenate suffix tokens to input_ids for the complete sequence
            full_input_ids = []
            full_attention_mask = []
            full_token_type_ids = []

            for i in range(batch_size):
                prompt_len = inputs["attention_mask"][i].sum().item()
                suffix_len = suffix_encodings["attention_mask"][i].sum().item()

                # Combine input_ids
                combined_input_ids = torch.cat([
                    inputs["input_ids"][i, :prompt_len],
                    suffix_encodings["input_ids"][i, 1:suffix_len]  # Skip BOS token
                ])

                # Create attention mask
                combined_attention_mask = torch.ones(combined_input_ids.shape[0])

                # Create token_type_ids (0 for prompt, 1 for suffix)
                combined_token_type_ids = torch.cat([
                    torch.zeros(prompt_len),
                    torch.ones(suffix_len - 1)  # Skip BOS token
                ])

                # Create labels (only for suffix part)
                combined_labels = torch.cat([
                    torch.full((prompt_len,), fill_value=-100),
                    suffix_encodings["input_ids"][i, 1:suffix_len]  # Skip BOS token
                ])

                # Pad to max length if needed
                max_len = max(seq_len, prompt_len + suffix_len - 1)

                padded_input_ids = torch.full((max_len,), fill_value=self.tokenizer.pad_token_id)
                padded_attention_mask = torch.zeros(max_len)
                padded_token_type_ids = torch.zeros(max_len)
                padded_labels = torch.full((max_len,), fill_value=-100)

                combined_len = combined_input_ids.shape[0]
                padded_input_ids[:combined_len] = combined_input_ids
                padded_attention_mask[:combined_len] = combined_attention_mask
                padded_token_type_ids[:combined_len] = combined_token_type_ids
                padded_labels[:combined_len] = combined_labels

                full_input_ids.append(padded_input_ids)
                full_attention_mask.append(padded_attention_mask)
                full_token_type_ids.append(padded_token_type_ids)

            # Stack tensors
            inputs["input_ids"] = torch.stack(full_input_ids)
            inputs["attention_mask"] = torch.stack(full_attention_mask)
            inputs["token_type_ids"] = torch.stack(full_token_type_ids)
            inputs["labels"] = torch.stack(full_input_ids)  # Use input_ids as labels

            # Replace padding token ids with -100 in labels
            inputs["labels"][inputs["labels"] == self.tokenizer.pad_token_id] = -100

        else:
            # Standard processing without suffix
            inputs = self.tokenizer(
                input_strings,
                return_tensors=return_tensors,
                padding=padding,
                truncation=truncation,
            )

        return_data = {"pixel_values": pixel_values, **inputs}
        return return_data



# === inference.py ===
from PIL import Image
import torch
import fire

def move_inputs_to_device(model_inputs: dict, device: str):
    model_inputs = {k: v.to(device) for k, v in model_inputs.items()}
    return model_inputs


def get_model_inputs(
    processor: PaliGemmaProcessor, prompt: str, image_file_path: str, device: str
):
    image = Image.open(image_file_path)
    images = [image]
    prompts = [prompt]
    model_inputs = processor(text=prompts, images=images)
    model_inputs = move_inputs_to_device(model_inputs, device)
    return model_inputs


def test_inference(
    model: PaliGemmaForConditionalGeneration,
    processor: PaliGemmaProcessor,
    device: str,
    prompt: str,
    image_file_path: str,
    max_tokens_to_generate: int,
    temperature: float,
    top_p: float,
    do_sample: bool,
):
    model_inputs = get_model_inputs(processor, prompt, image_file_path, device)
    input_ids = model_inputs["input_ids"]
    attention_mask = model_inputs["attention_mask"]
    pixel_values = model_inputs["pixel_values"]

    kv_cache = KVCache()

    # Generate tokens until you see the stop token
    stop_token = processor.tokenizer.eos_token_id
    generated_tokens = []

    for _ in range(max_tokens_to_generate):
        # Get the model outputs
        # TODO: remove the labels
        outputs = model(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            kv_cache=kv_cache,
        )
        kv_cache = outputs["kv_cache"]
        next_token_logits = outputs["logits"][:, -1, :]
        # Sample the next token
        if do_sample:
            # Apply temperature
            next_token_logits = torch.softmax(next_token_logits / temperature, dim=-1)
            next_token = _sample_top_p(next_token_logits, top_p)
        else:
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
        assert next_token.size() == (1, 1)
        next_token = next_token.squeeze(0)  # Remove batch dimension
        generated_tokens.append(next_token)
        # Stop if the stop token has been generated
        if next_token.item() == stop_token:
            break
        # Append the next token to the input
        input_ids = next_token.unsqueeze(-1)
        attention_mask = torch.cat(
            [attention_mask, torch.ones((1, 1), device=input_ids.device)], dim=-1
        )

    generated_tokens = torch.cat(generated_tokens, dim=-1)
    # Decode the generated tokens
    decoded = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)

    print(prompt + decoded)


def _sample_top_p(probs: torch.Tensor, p: float):
    # (B, vocab_size)
    probs_sort, probs_idx = torch.sort(probs, dim=-1, descending=True)
    # (B, vocab_size)
    probs_sum = torch.cumsum(probs_sort, dim=-1)
    # (B, vocab_size)
    # (Substracting "probs_sort" shifts the cumulative sum by 1 position to the right before masking)
    mask = probs_sum - probs_sort > p
    # Zero out all the probabilities of tokens that are not selected by the Top P
    probs_sort[mask] = 0.0
    # Redistribute the probabilities so that they sum up to 1.
    probs_sort.div_(probs_sort.sum(dim=-1, keepdim=True))
    # Sample a token (its index) from the top p distribution
    next_token = torch.multinomial(probs_sort, num_samples=1)
    # Get the token position in the vocabulary corresponding to the sampled index
    next_token = torch.gather(probs_idx, -1, next_token)
    return next_token

def main(
    model_path: str = "/content/Files",
    prompts: list = None,  # Change this to accept a list of prompts
    image_file_path: str = "/content/2341.jpg",
    max_tokens_to_generate: int = 100,
    temperature: float = 0.8,
    top_p: float = 0.9,
    do_sample: bool = False,
    only_cpu: bool = False,
):
    device = "cpu"

    if not only_cpu:
        if torch.cuda.is_available():
            device = "cuda"
        elif torch.backends.mps.is_available():
            device = "mps"

    print("Device in use: ", device)

    print(f"Loading model")
    model, tokenizer = load_hf_model(model_path, device)
    model = model.to(device).eval()

    num_image_tokens = model.config.vision_config.num_image_tokens
    image_size = model.config.vision_config.image_size
    processor = PaliGemmaProcessor(tokenizer, num_image_tokens, image_size)

    print("Running inference for multiple prompts:")
    with torch.no_grad():
        # Iterate over all prompts and run inference
        for prompt in prompts:
            print(f"Processing prompt: {prompt}")
            test_inference(
                model,
                processor,
                device,
                prompt,
                image_file_path,
                max_tokens_to_generate,
                temperature,
                top_p,
                do_sample,
            )

if __name__ == "__main__":
    # Check if we are in a Colab environment and bypass `fire` if it's a notebook
    try:
        import google.colab
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
       # Example usage with multiple prompts
        prompts = [
          "What is the subcategory of this fashion item?",
          "What type of article is this fashion item?",
          "What is the base color of this fashion item?",
          "What season is this fashion item designed for?",
          "What is the intended usage of this fashion item?"
        ]
        model_path = "/content/Files"  # Or any path you use
        image_file_path = "/content/2341.jpg"  # Update this with the correct path
        max_tokens_to_generate = 100
        temperature = 0.9
        top_p = 0.4
        do_sample = False
        only_cpu = False

        main(
            model_path=model_path,
            prompts=prompts,  # Pass the list of prompts here
            image_file_path=image_file_path,
            max_tokens_to_generate=max_tokens_to_generate,
            temperature=temperature,
            top_p=top_p,
            do_sample=do_sample,
            only_cpu=only_cpu,
        )
    else:
        # For other environments (like terminal), use fire
        import fire
        fire.Fire(main)

Device in use:  cpu
Loading model


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Running inference for multiple prompts:
Processing prompt: What is the subcategory of this fashion item?
What is the subcategory of this fashion item?clothing
Processing prompt: What type of article is this fashion item?
What type of article is this fashion item?all
Processing prompt: What is the base color of this fashion item?
What is the base color of this fashion item?black
Processing prompt: What season is this fashion item designed for?
What season is this fashion item designed for?summer
Processing prompt: What is the intended usage of this fashion item?
What is the intended usage of this fashion item?main


# combined application

In [ ]:
import os
import zipfile
import torch
import tiktoken
import json
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from google.colab import drive

# Define paths
MODEL_PATH = "/content/Files"  # Base VLM model path
TRAINED_VLM_PATH = "/content/drive/MyDrive/finetuned_paligemma/temp_train_model/model.pt"  # Fine-tuned VLM path
LLM_ZIP_PATH = '/content/drive/MyDrive/fashion_files/best_model.zip'  # LLM model zip path
LLM_UNZIP_DESTINATION = '/content/extracted_model/'  # Where to extract LLM model
USER_TEXT_PATH = '/content/drive/MyDrive/fashion_files/user.txt'  # User profile path
CONFIG_PATH = '/content/drive/MyDrive/fashion_files/generation_config.json'  # Generation config path
OUTPUT_DIR = "/content/drive/MyDrive/finetuned_paligemma"  # Output directory

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Default generation parameters (will be overridden by config file if present)
DEFAULT_CONFIG = {
    "max_new_tokens": 150,
    "temperature": 0.7,
    "top_k": 40,
    "instruction": "",
    "input_text": "",
    "consider_wardrobe": "no",  # For personalization
    "consider_user_details": "no"  # For personalization
}

# Ensure directories exist
os.makedirs(LLM_UNZIP_DESTINATION, exist_ok=True)
os.makedirs(os.path.dirname(USER_TEXT_PATH), exist_ok=True)

# PART 1: VLM FUNCTIONS
# -------------------------------------

# Function to capture images from webcam in Colab
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => {
            capture.onclick = resolve;
          });

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

def load_finetuned_vlm(base_model_path, trained_model_path, device):
    print(f"Loading base VLM model from {base_model_path}")

    # Import model architecture and processor
    # Note: These would need to be properly imported in a real implementation
    # from paligemma import load_hf_model, PaliGemmaProcessor, KVCache

    model, tokenizer = load_hf_model(base_model_path, device)
    if os.path.exists(trained_model_path):
        print(f"Loading fine-tuned VLM weights from {trained_model_path}")
        state_dict = torch.load(trained_model_path, map_location=device)
        model.load_state_dict(state_dict, strict=False)
        print("Successfully loaded fine-tuned VLM weights")
    else:
        print(f"Warning: Fine-tuned VLM model not found at {trained_model_path}. Using base model.")
    model.eval()
    return model, tokenizer

def move_inputs_to_device(model_inputs: dict, device: str):
    return {k: v.to(device) for k, v in model_inputs.items()}

def get_model_inputs(processor, prompt, image, device):
    """Process image (as numpy array or PIL Image) with the processor"""
    if isinstance(image, np.ndarray):
        # Convert OpenCV BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image_rgb)
    else:
        pil_image = image

    model_inputs = processor(text=[prompt], images=[pil_image])
    model_inputs = move_inputs_to_device(model_inputs, device)
    return model_inputs

def generate_vlm_prediction(model, processor, image, prompt, max_tokens=50):
    """Generate prediction for an image (can be PIL Image or numpy array)"""
    model_inputs = get_model_inputs(processor, prompt, image, device)
    input_ids = model_inputs["input_ids"]
    attention_mask = model_inputs["attention_mask"]
    pixel_values = model_inputs["pixel_values"]
    kv_cache = KVCache()
    stop_token = processor.tokenizer.eos_token_id
    generated_tokens = []

    with torch.no_grad():
        for _ in range(max_tokens):
            # Always pass pixel_values for this architecture
            outputs = model(
                input_ids=input_ids,
                pixel_values=pixel_values,
                attention_mask=attention_mask,
                kv_cache=kv_cache,
            )
            kv_cache = outputs["kv_cache"]
            next_token_logits = outputs["logits"][:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            token_id = next_token.item()
            generated_tokens.append(token_id)
            if token_id == stop_token:
                break
            input_ids = next_token
            attention_mask = torch.cat([attention_mask, torch.ones((1, 1), device=device)], dim=-1)

    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

def process_image_with_vlm(vlm_model, processor, image_path=None, use_webcam=False):
    """Process an image with the VLM and return formatted results"""
    # Define the prompts for the VLM
    prompts = [
        "What is the subcategory of this fashion item?",
        "What type of article is this fashion item?",
        "What is the base color of this fashion item?",
        "What season is this fashion item designed for?",
        "What is the intended usage of this fashion item?"
    ]

    # Get image from webcam or file
    if use_webcam:
        print("Click 'Capture' to take a photo for fashion analysis")
        filename = take_photo()
        print(f"Photo saved as {filename}")
        img = Image.open(filename)
    else:
        # Use provided image path
        if not os.path.exists(image_path):
            print(f"Image not found at {image_path}")
            return None
        img = Image.open(image_path)

    # Display the image
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.axis('off')
    plt.title("Fashion Item")
    plt.show()

    # Process the image with VLM
    print("\nRunning VLM analysis on image...")
    results = {}
    for prompt in prompts:
        prediction = generate_vlm_prediction(vlm_model, processor, img, prompt)
        # Store the result
        key = prompt.replace("What is the ", "").replace("?", "").strip()
        results[key] = prediction

    # Create a concise description
    concise_desc = f"given image: {results.get('intended usage', '').lower()} " \
                  f"{results.get('base color', '').lower()} " \
                  f"{results.get('type of article', '').lower()} " \
                  f"{results.get('subcategory', '').lower()} " \
                  f"for {results.get('season', '').lower()}"

    # Display detailed results
    print("\nVLM Analysis Results:")
    for prompt, result in zip(prompts, results.values()):
        print(f"{prompt} {result}")

    print(f"\nConcise description: {concise_desc}")

    return concise_desc


# PART 2: LLM FUNCTIONS
# -------------------------------------

# Class for parsing and handling user profile data
class UserProfileManager:
    def __init__(self, user_text_path):
        self.user_text_path = user_text_path
        self.user_data = {}
        self.user_wardrobe = []
        self.load_user_data()

    def load_user_data(self):
        """Load and parse user data from user.txt"""
        if not os.path.exists(self.user_text_path):
            print(f"User profile not found at {self.user_text_path}")
            return False

        try:
            with open(self.user_text_path, 'r') as f:
                content = f.read().strip()
                # Parse the user data
                self._parse_user_data(content)
                print(f"Loaded user profile from {self.user_text_path}")
                return True
        except Exception as e:
            print(f"Error loading user profile: {e}")
            return False

    def _parse_user_data(self, content):
        """Parse user data content into structured data"""
        lines = content.split('\n')
        for line in lines:
            # Skip empty lines and lines without a proper key:value format
            if not line.strip() or ':' not in line:
                continue

            parts = line.split(':', 1)
            key = parts[0].strip().lower()
            value = parts[1].strip()

            # Special handling for wardrobe items
            if key == 'wardrobe':
                self.user_wardrobe = [item.strip() for item in value.split(',')]
                self.user_data[key] = self.user_wardrobe
            else:
                self.user_data[key] = value

    def get_user_profile_summary(self):
        """Get a formatted summary of the user profile for the model"""
        if not self.user_data:
            return "No user profile data available."

        summary = []
        # Add basic user information
        for key, value in self.user_data.items():
            if key != 'wardrobe':  # Handle wardrobe separately
                summary.append(f"{key}: {value}")

        return "\n".join(summary)

    def get_wardrobe_summary(self):
        """Get a formatted summary of the user's wardrobe"""
        if not self.user_wardrobe:
            return "No wardrobe information available."

        return "Wardrobe items: " + ", ".join(self.user_wardrobe)

# Function to extract zip with progress tracking
def extract_with_progress(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Get file info
        file_info_list = zip_ref.infolist()

        # Set up tqdm progress bar
        with tqdm(total=len(file_info_list), desc="Extracting files") as pbar:
            for file_info in file_info_list:
                zip_ref.extract(file_info, extract_to)
                pbar.update(1)

    print(f"Extraction complete to {extract_to}")
    return os.path.join(extract_to, os.path.basename(zip_path).replace('.zip', '.pt'))

def format_input(instruction, input_text="", consider_wardrobe="no", consider_user_details="no", user_profile=None):
    """Format the instruction and input according to the training format, including personalization options."""
    # Base instruction
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request. "
        f"Do not generate additional inputs or profile information."
        f"\n\n### Instruction:\n{instruction}"
    )

    # Add personalization details if requested
    if user_profile and consider_user_details.lower() == "yes":
        instruction_text += f"\n\nConsider the following user details for personalization:\n{user_profile.get_user_profile_summary()}"

    # Add wardrobe information if requested
    if user_profile and consider_wardrobe.lower() == "yes":
        instruction_text += f"\n\nConsider these items from the user's wardrobe for your suggestions:\n{user_profile.get_wardrobe_summary()}"

    # Add input text if provided
    input_part = f"\n\n### Input:\n{input_text}" if input_text else ""

    return instruction_text + input_part + "\n\n### Response:"

def load_finetuned_llm(checkpoint_path, device):
    """Load the fine-tuned LLM from a checkpoint."""
    print(f"Loading LLM model from {checkpoint_path}")

    # Import your model definitions here
    # from gpt_model import GPTModel, GPT_CONFIG_774M

    # Initialize model with configuration
    model = GPTModel(GPT_CONFIG_774M)

    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.load_state_dict(checkpoint["model_state_dict"])

    # Move model to device
    model.to(device)
    model.eval()

    print(f"LLM model loaded successfully! Validation loss: {checkpoint.get('val_loss', 'N/A')}")
    return model

def generate_llm_text(model, tokenizer, prompt, device, max_new_tokens=150, temperature=0.7, top_k=40):
    """Generate text based on a prompt using the fine-tuned LLM."""
    # Tokenize the prompt
    encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)
    context_size = model.pos_emb.weight.shape[0]

    # Generate response
    model.eval()

    print(f"Generation parameters: max_tokens={max_new_tokens}, temp={temperature}, top_k={top_k}")
    generated_tokens = []

    with torch.no_grad():
        for _ in tqdm(range(max_new_tokens), desc="Generating tokens"):
            # Get context for the current step
            idx_cond = encoded[:, -context_size:]

            # Forward pass
            logits = model(idx_cond)

            # Get logits for the last token
            logits = logits[:, -1, :]

            # Apply temperature and top-k filtering
            if temperature > 0:
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = float('-inf')

                probs = torch.softmax(logits / temperature, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)

            # Append new token
            encoded = torch.cat((encoded, idx_next), dim=1)
            generated_tokens.append(idx_next.item())

            # Stop if EOS token is generated (50256 is <|endoftext|>)
            if idx_next.item() == 50256:
                break

            # Also stop if we see "### Input:" - this is a sign the model is hallucinating additional input
            current_text = tokenizer.decode(encoded.squeeze().tolist()[len(tokenizer.encode(prompt)):])
            if "### Input:" in current_text:
                break

    # Decode the generated text
    generated_text = tokenizer.decode(encoded.squeeze().tolist())

    # Return only the response part (after the prompt)
    response = generated_text[len(prompt):]

    # Remove any hallucinated input sections
    if "### Input:" in response:
        response = response.split("### Input:")[0].strip()

    # Remove any hallucinated response markers
    if "### Response:" in response:
        response = response.replace("### Response:", "").strip()

    return response, generated_tokens

def load_user_config():
    """Load user configuration from config file if it exists, otherwise use defaults."""
    config = DEFAULT_CONFIG.copy()

    # Check if config file exists
    if os.path.exists(CONFIG_PATH):
        try:
            with open(CONFIG_PATH, 'r') as f:
                user_config = json.load(f)
                config.update(user_config)
                print(f"Loaded configuration from {CONFIG_PATH}")
        except Exception as e:
            print(f"Error loading config file: {e}")
            print("Using default configuration")
    else:
        print(f"Config file not found at {CONFIG_PATH}. Using default configuration.")
        # Create a template config file for future use
        try:
            with open(CONFIG_PATH, 'w') as f:
                json.dump(DEFAULT_CONFIG, f, indent=4)
                print(f"Created template config file at {CONFIG_PATH}")
        except Exception as e:
            print(f"Failed to create template config file: {e}")

    return config

def update_config_interactive(config):
    """Allow user to interactively update configuration parameters."""
    print("\n===== Generation Configuration =====")
    print("Current settings:")
    for key, value in config.items():
        print(f"  {key}: {value}")

    print("\nEnter new values (press Enter to keep current value):")

    # Update numerical parameters
    for param in ["max_new_tokens", "temperature", "top_k"]:
        new_value = input(f"{param} [{config[param]}]: ")
        if new_value.strip():
            try:
                if param == "max_new_tokens" or param == "top_k":
                    config[param] = int(new_value)
                else:
                    config[param] = float(new_value)
            except ValueError:
                print(f"Invalid value for {param}, keeping current value.")

    # Update personalization parameters
    for param in ["consider_wardrobe", "consider_user_details"]:
        new_value = input(f"{param} [{config[param]}] (yes/no): ")
        if new_value.strip().lower() in ['yes', 'no']:
            config[param] = new_value.lower()

    # Save updated config
    try:
        with open(CONFIG_PATH, 'w') as f:
            json.dump(config, f, indent=4)
            print(f"Configuration saved to {CONFIG_PATH}")
    except Exception as e:
        print(f"Failed to save configuration: {e}")

    return config

# PART 3: USER PROFILE MANAGEMENT
# -------------------------------------

def update_user_profile(user_profile):
    """Interactive function to update user profile data"""
    print("\n===== Update User Profile =====")

    if user_profile.user_data:
        print("Current profile:")
        for key, value in user_profile.user_data.items():
            if key != 'wardrobe':  # Display wardrobe separately
                print(f"  {key}: {value}")

        # Display wardrobe items separately
        if user_profile.user_wardrobe:
            print("Wardrobe items:")
            for i, item in enumerate(user_profile.user_wardrobe, 1):
                print(f"  {i}. {item}")
    else:
        print("No profile data found. Creating new profile.")

    print("\nOptions:")
    print("1. Edit basic profile information")
    print("2. Edit wardrobe items")
    print("3. Cancel")

    choice = input("Choose an option (1-3): ")

    if choice == '1':
        # Edit basic profile info
        print("\nEnter profile information (press Enter to keep current values, 'delete' to remove):")
        fields = ['name', 'age', 'gender', 'location', 'style', 'occupation', 'preferences', 'favorite_colors', 'size']

        updated_data = {}
        for field in fields:
            current = user_profile.user_data.get(field, '')
            new_value = input(f"{field} [{current}]: ")

            if new_value.lower() == 'delete':
                # Don't include this field
                pass
            elif new_value.strip():
                # Update with new value
                updated_data[field] = new_value
            elif field in user_profile.user_data:
                # Keep current value
                updated_data[field] = current

        # Preserve wardrobe data
        if 'wardrobe' in user_profile.user_data:
            updated_data['wardrobe'] = user_profile.user_data['wardrobe']

        # Update user data
        user_profile.user_data = updated_data

        # Save to file
        save_updated_profile(user_profile)

    elif choice == '2':
        # Edit wardrobe items
        print("\nOptions for wardrobe:")
        print("1. Add items")
        print("2. Remove items")
        print("3. Replace all items")
        print("4. Cancel")

        wardrobe_choice = input("Choose an option (1-4): ")

        if wardrobe_choice == '1':
            # Add items
            new_items = input("Enter items to add (comma separated): ")
            items_to_add = [item.strip() for item in new_items.split(',') if item.strip()]

            if 'wardrobe' not in user_profile.user_data:
                user_profile.user_data['wardrobe'] = []
                user_profile.user_wardrobe = []

            user_profile.user_wardrobe.extend(items_to_add)
            user_profile.user_data['wardrobe'] = user_profile.user_wardrobe

            save_updated_profile(user_profile)

        elif wardrobe_choice == '2':
            # Remove items
            if not user_profile.user_wardrobe:
                print("No wardrobe items to remove.")
                return

            print("Current items:")
            for i, item in enumerate(user_profile.user_wardrobe, 1):
                print(f"  {i}. {item}")

            to_remove = input("Enter numbers of items to remove (comma separated): ")
            try:
                indices = [int(idx.strip()) for idx in to_remove.split(',') if idx.strip()]
                # Sort in reverse to avoid index shifting during removal
                indices.sort(reverse=True)

                for idx in indices:
                    if 1 <= idx <= len(user_profile.user_wardrobe):
                        del user_profile.user_wardrobe[idx-1]

                user_profile.user_data['wardrobe'] = user_profile.user_wardrobe
                save_updated_profile(user_profile)

            except ValueError:
                print("Invalid input. Please enter numbers separated by commas.")

        elif wardrobe_choice == '3':
            # Replace all
            new_wardrobe = input("Enter new wardrobe items (comma separated): ")
            new_items = [item.strip() for item in new_wardrobe.split(',') if item.strip()]

            user_profile.user_wardrobe = new_items
            user_profile.user_data['wardrobe'] = new_items

            save_updated_profile(user_profile)

def save_updated_profile(user_profile):
    """Save updated profile to user.txt file"""
    try:
        # Format profile data for saving
        lines = []

        # Add basic profile fields first
        for key, value in user_profile.user_data.items():
            if key != 'wardrobe':  # Handle wardrobe separately
                lines.append(f"{key}: {value}")

        # Add wardrobe at the end
        if user_profile.user_wardrobe:
            lines.append(f"wardrobe: {', '.join(user_profile.user_wardrobe)}")

        # Write to file
        with open(user_profile.user_text_path, 'w') as f:
            f.write('\n'.join(lines))

        print(f"Profile updated and saved to {user_profile.user_text_path}")
        # Reload the profile to ensure everything is up to date
        user_profile.load_user_data()
        return True

    except Exception as e:
        print(f"Error saving profile: {e}")
        return False

# PART 4: INTEGRATED SYSTEM
# -------------------------------------

def run_integrated_fashion_system():
    """Main function to run the integrated fashion recommendation system"""
    # Mount Google Drive if needed
    if not os.path.exists("/content/drive"):
        drive.mount('/content/drive')

    # Step 1: Extract LLM model
    print("\n===== Loading Models =====")
    llm_model_path = extract_with_progress(LLM_ZIP_PATH, LLM_UNZIP_DESTINATION)

    # Step 2: Load tokenizer for LLM
    tokenizer = tiktoken.get_encoding("gpt2")

    # Step 3: Load user profile
    user_profile = UserProfileManager(USER_TEXT_PATH)

    # Check if user profile needs to be created or updated
    if not user_profile.user_data:
        print("No user profile found. Would you like to create one? (y/n)")
        if input().lower() == 'y':
            update_user_profile(user_profile)
    else:
        print("User profile loaded. Would you like to update it? (y/n)")
        if input().lower() == 'y':
            update_user_profile(user_profile)

    # Step 4: Load models
    llm_model = load_finetuned_llm(llm_model_path, device)

    # Main loop
    while True:
        print("\n===== Integrated Fashion Recommendation System =====")
        print("Options:")
        print("1. Use fashion image for recommendation")
        print("2. Get fashion advice without image")
        print("3. Update user profile")
        print("4. Exit")

        option = input("Choose an option (1-4): ")

        if option == '1':
            # Use fashion image
            image_input_type = input("Do you want to (1) use webcam or (2) upload an image file? ")

            image_description = None
            if image_input_type == '1':
                # Load VLM only when needed (since it might be large)
                vlm_model, vlm_tokenizer = load_finetuned_vlm(MODEL_PATH, TRAINED_VLM_PATH, device)
                num_image_tokens = vlm_model.config.vision_config.num_image_tokens
                image_size = vlm_model.config.vision_config.image_size
                processor = PaliGemmaProcessor(vlm_tokenizer, num_image_tokens, image_size)

                # Use webcam
                image_description = process_image_with_vlm(vlm_model, processor, use_webcam=True)
            elif image_input_type == '2':
                # Upload file
                image_path = input("Enter the path to the image file: ")

                # Load VLM only when needed
                vlm_model, vlm_tokenizer = load_finetuned_vlm(MODEL_PATH, TRAINED_VLM_PATH, device)
                num_image_tokens = vlm_model.config.vision_config.num_image_tokens
                image_size = vlm_model.config.vision_config.image_size
                processor = PaliGemmaProcessor(vlm_tokenizer, num_image_tokens, image_size)

                image_description = process_image_with_vlm(vlm_model, processor, image_path=image_path)
            else:
                print("Invalid option. Please try again.")
                continue

            if not image_description:
                print("Failed to analyze image. Please try again.")
                continue

            # Now get instruction from user
            instruction = input("\nWhat would you like to know about this fashion item? ")

            # Load config for LLM generation
            config = load_user_config()

            # Allow user to adjust generation parameters
            print("\nWould you like to adjust generation parameters? (y/n)")
            if input().lower() == 'y':
                config = update_config_interactive(config)

            # Format the instruction with the image description
            combined_input = f"{image_description}\n\n{instruction}"

            # Format the prompt with personalization options
            prompt = format_input(
                instruction,
                combined_input,
                config["consider_wardrobe"],
                config["consider_user_details"],
                user_profile
            )

            # Generate response
            print("\nGenerating fashion advice...")
            response, tokens = generate_llm_text(
                llm_model, tokenizer, prompt, device,
                max_new_tokens=config["max_new_tokens"],
                temperature=config["temperature"],
                top_k=config["top_k"]
            )

            # Display results
            print("\n=== Fashion Advice ===")
            print(response)
            print("\n=== Generation Stats ===")
            print(f"Generated {len(tokens)} tokens")
            print(f"Personalization: Wardrobe = {config['consider_wardrobe']}, User Details = {config['consider_user_details']}")

        elif option == '2':
            # Get fashion advice without image
            # Get fashion advice without image
            instruction = input("\nWhat fashion advice would you like? ")

            # Load config
            config = load_user_config()

            # Allow user to adjust generation parameters
            print("\nWould you like to adjust generation parameters? (y/n)")
            if input().lower() == 'y':
                config = update_config_interactive(config)

            # Optional input text
            input_text = ""  # No additional input for text-only mode

            # Format the prompt with personalization options
            prompt = format_input(
                instruction,
                input_text,
                config["consider_wardrobe"],
                config["consider_user_details"],
                user_profile
            )

            # Generate response
            print("\nGenerating fashion advice...")
            response, tokens = generate_llm_text(
                llm_model, tokenizer, prompt, device,
                max_new_tokens=config["max_new_tokens"],
                temperature=config["temperature"],
                top_k=config["top_k"]
            )

            # Display results
            print("\n=== Fashion Advice ===")
            print(response)
            print("\n=== Generation Stats ===")
            print(f"Generated {len(tokens)} tokens")
            print(f"Personalization: Wardrobe = {config['consider_wardrobe']}, User Details = {config['consider_user_details']}")

        elif option == '3':
            # Update user profile
            update_user_profile(user_profile)

        elif option == '4':
            # Exit
            print("Thank you for using the Fashion Recommendation System. Goodbye!")
            break

        else:
            print("Invalid option. Please try again.")

        # Ask if user wants to continue
        print("\nPress Enter to continue...")
        input()


if __name__ == "__main__":
    run_integrated_fashion_system()

Using device: cpu

===== Loading Models =====


Extracting files: 100%|██████████| 1/1 [02:51<00:00, 171.58s/it]


Extraction complete to /content/extracted_model/
Loaded user profile from /content/drive/MyDrive/fashion_files/user.txt
User profile loaded. Would you like to update it? (y/n)
y

===== Update User Profile =====
Current profile:
  name: sriram
  age: 21
  gender: male
  location: india
  style: minimalist
  occupation: student
  preferences: minimalistic, stylish, classy
  favorite_colors: navy, beige, white, black
  size: m
Wardrobe items:
  1. white button-up shirt
  2. navy trousers
  3. black coat
  4. beige cashmere sweater
  5. white sneakers
  6. black ankle boots
  7. denim jeans
  8. white t-shirts
  9. pink polo tshirt
  10. little black dress
  11. brown watch
  12. cotton black tshirt
  13. brown trousers

Options:
1. Edit basic profile information
2. Edit wardrobe items
3. Cancel
Choose an option (1-3): 3
Loading LLM model from /content/extracted_model/best_model.pt
LLM model loaded successfully! Validation loss: 1.2289036760727565

===== Integrated Fashion Recommendation S

<IPython.core.display.Javascript object>

MessageError: NotAllowedError: Permission denied

with tkinter

In [ ]:
import os
import zipfile
import json
import threading
import tkinter as tk
from tkinter import ttk, filedialog, scrolledtext, messagebox
from PIL import Image, ImageTk
import torch
import tiktoken
import numpy as np
import cv2
from tqdm import tqdm


# Define paths
MODEL_PATH = "/content/Files"  # Base VLM model path
TRAINED_VLM_PATH = "/content/drive/MyDrive/finetuned_paligemma/temp_train_model/model.pt"  # Fine-tuned VLM path
LLM_ZIP_PATH = '/content/drive/MyDrive/fashion_files/best_model.zip'  # LLM model zip path
LLM_EXTRACT_DIR = '/content/extracted_model/'  # Where to extract LLM model
USER_PROFILE_PATH = '/content/drive/MyDrive/fashion_files/user.txt'  # User profile path
CONFIG_PATH = '/content/drive/MyDrive/fashion_files/generation_config.json'  # Generation config path
OUTPUT_DIR = "/content/drive/MyDrive/finetuned_paligemma"  # Output directory

# Default generation parameters
DEFAULT_CONFIG = {
    "max_new_tokens": 150,
    "temperature": 0.7,
    "top_k": 40,
    "consider_wardrobe": "no",  # For personalization
    "consider_user_details": "no"  # For personalization
}

# Global variables for models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llm_model = None
vlm_model = None
vlm_processor = None
tokenizer = None
user_profile = None
webcam = None


# ---------- MODEL MANAGER ----------
class ModelManager:
    """Class to manage model loading and initialization"""

    def __init__(self, status_callback=None):
        self.llm_model = None
        self.vlm_model = None
        self.vlm_processor = None
        self.tokenizer = None
        self.status_callback = status_callback
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def update_status(self, message):
        """Update status message if callback is provided"""
        if self.status_callback:
            self.status_callback(message)
        else:
            print(message)

    def initialize_models(self):
        """Initialize all required models"""
        try:
            # Initialize tokenizer first (lightweight)
            self.update_status("Initializing tokenizer...")
            self.tokenizer = tiktoken.get_encoding("gpt2")

            # Load models in separate threads to improve startup time
            llm_thread = threading.Thread(target=self._load_llm)
            vlm_thread = threading.Thread(target=self._load_vlm)

            self.update_status("Loading models in background...")
            llm_thread.start()
            vlm_thread.start()

            # Wait for models to load
            llm_thread.join()
            vlm_thread.join()

            self.update_status("All models loaded successfully!")
            return True
        except Exception as e:
            self.update_status(f"Failed to initialize models: {str(e)}")
            return False

    def _load_llm(self):
        """Load LLM model"""
        try:
            self.update_status("Loading LLM model...")

            # Extract LLM if needed
            llm_model_path = self._extract_model_if_needed(LLM_ZIP_PATH, LLM_EXTRACT_DIR)

            # Import required modules here to avoid global imports
            from gpt_model import GPTModel, GPT_CONFIG_774M

            # Initialize model with configuration
            model = GPTModel(GPT_CONFIG_774M)

            # Load the checkpoint
            checkpoint = torch.load(llm_model_path, map_location=self.device)
            model.load_state_dict(checkpoint["model_state_dict"])

            # Move model to device
            model.to(self.device)
            model.eval()

            self.llm_model = model
            self.update_status("LLM model loaded successfully!")
        except Exception as e:
            self.update_status(f"Error loading LLM: {str(e)}")
            raise

    def _load_vlm(self):
        """Load VLM model"""
        try:
            self.update_status("Loading VLM model...")

            # Import model architecture and processor
            from paligemma import load_hf_model, PaliGemmaProcessor

            # Load base model
            model, tokenizer = load_hf_model(MODEL_PATH, self.device)

            # Load fine-tuned weights if available
            if os.path.exists(TRAINED_VLM_PATH):
                self.update_status("Loading fine-tuned VLM weights...")
                state_dict = torch.load(TRAINED_VLM_PATH, map_location=self.device)
                model.load_state_dict(state_dict, strict=False)

            model.eval()

            # Initialize processor
            num_image_tokens = model.config.vision_config.num_image_tokens
            image_size = model.config.vision_config.image_size
            processor = PaliGemmaProcessor(tokenizer, num_image_tokens, image_size)

            self.vlm_model = model
            self.vlm_processor = processor
            self.update_status("VLM model loaded successfully!")
        except Exception as e:
            self.update_status(f"Error loading VLM: {str(e)}")
            raise

    def _extract_model_if_needed(self, zip_path, extract_dir):
        """Extract model zip if needed"""
        # Expected path for the extracted model
        model_path = os.path.join(extract_dir, os.path.basename(zip_path).replace('.zip', '.pt'))

        # Check if model is already extracted
        if os.path.exists(model_path):
            self.update_status(f"Using existing model at {model_path}")
            return model_path

        # Create extraction directory if it doesn't exist
        os.makedirs(extract_dir, exist_ok=True)

        # Extract with progress tracking
        self.update_status(f"Extracting model from {zip_path}...")
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # Get file info
            file_info_list = zip_ref.infolist()

            # Extract all files
            for file_info in file_info_list:
                zip_ref.extract(file_info, extract_dir)

        self.update_status(f"Extraction complete to {extract_dir}")
        return model_path

    def get_models(self):
        """Get loaded models"""
        return {
            "llm": self.llm_model,
            "vlm": self.vlm_model,
            "processor": self.vlm_processor,
            "tokenizer": self.tokenizer,
            "device": self.device
        }


# ---------- USER PROFILE MANAGER ----------
class UserProfileManager:
    """Manager for user profile data"""

    def __init__(self, user_text_path):
        self.user_text_path = user_text_path
        self.user_data = {}
        self.user_wardrobe = []
        self.load_user_data()

    def load_user_data(self):
        """Load and parse user data from user.txt"""
        if not os.path.exists(self.user_text_path):
            print(f"User profile not found at {self.user_text_path}")
            return False

        try:
            with open(self.user_text_path, 'r') as f:
                content = f.read().strip()
                # Parse the user data
                self._parse_user_data(content)
                print(f"Loaded user profile from {self.user_text_path}")
                return True
        except Exception as e:
            print(f"Error loading user profile: {e}")
            return False

    def _parse_user_data(self, content):
        """Parse user data content into structured data"""
        lines = content.split('\n')
        for line in lines:
            # Skip empty lines and lines without a proper key:value format
            if not line.strip() or ':' not in line:
                continue

            parts = line.split(':', 1)
            key = parts[0].strip().lower()
            value = parts[1].strip()

            # Special handling for wardrobe items
            if key == 'wardrobe':
                self.user_wardrobe = [item.strip() for item in value.split(',')]
                self.user_data[key] = self.user_wardrobe
            else:
                self.user_data[key] = value

    def get_user_profile_summary(self):
        """Get a formatted summary of the user profile for the model"""
        if not self.user_data:
            return "No user profile data available."

        summary = []
        # Add basic user information
        for key, value in self.user_data.items():
            if key != 'wardrobe':  # Handle wardrobe separately
                summary.append(f"{key}: {value}")

        return "\n".join(summary)

    def get_wardrobe_summary(self):
        """Get a formatted summary of the user's wardrobe"""
        if not self.user_wardrobe:
            return "No wardrobe information available."

        return "Wardrobe items: " + ", ".join(self.user_wardrobe)

    def save_profile(self, user_data):
        """Save updated profile to file"""
        try:
            self.user_data = user_data
            if 'wardrobe' in user_data:
                self.user_wardrobe = user_data['wardrobe'] if isinstance(user_data['wardrobe'], list) else []

            # Format profile data for saving
            lines = []

            # Add basic profile fields first
            for key, value in self.user_data.items():
                if key != 'wardrobe':  # Handle wardrobe separately
                    lines.append(f"{key}: {value}")

            # Add wardrobe at the end
            if self.user_wardrobe:
                lines.append(f"wardrobe: {', '.join(self.user_wardrobe)}")

            # Write to file
            with open(self.user_text_path, 'w') as f:
                f.write('\n'.join(lines))

            print(f"Profile saved to {self.user_text_path}")
            return True
        except Exception as e:
            print(f"Error saving profile: {e}")
            return False


# ---------- CONFIG MANAGER ----------
class ConfigManager:
    """Manager for generation configuration"""

    def __init__(self, config_path):
        self.config_path = config_path
        self.config = DEFAULT_CONFIG.copy()
        self.load_config()

    def load_config(self):
        """Load configuration from file or create default"""
        # Check if config file exists
        if os.path.exists(self.config_path):
            try:
                with open(self.config_path, 'r') as f:
                    user_config = json.load(f)
                    self.config.update(user_config)
                    print(f"Loaded configuration from {self.config_path}")
            except Exception as e:
                print(f"Error loading config file: {e}")
                print("Using default configuration")
        else:
            print(f"Config file not found at {self.config_path}. Creating default.")
            self.save_config()

    def save_config(self):
        """Save current configuration to file"""
        try:
            with open(self.config_path, 'w') as f:
                json.dump(self.config, f, indent=4)
                print(f"Configuration saved to {self.config_path}")
            return True
        except Exception as e:
            print(f"Failed to save configuration: {e}")
            return False

    def get_config(self):
        """Get current configuration"""
        return self.config

    def update_config(self, new_config):
        """Update configuration with new values"""
        self.config.update(new_config)
        return self.save_config()


# ---------- ML FUNCTIONS ----------
def process_image_with_vlm(vlm_model, processor, image, device):
    """Process an image with the VLM and return formatted results"""
    # Define the prompts for the VLM
    prompts = [
        "What is the subcategory of this fashion item?",
        "What type of article is this fashion item?",
        "What is the base color of this fashion item?",
        "What season is this fashion item designed for?",
        "What is the intended usage of this fashion item?"
    ]

    # Process the image with VLM
    results = {}
    for prompt in prompts:
        prediction = generate_vlm_prediction(vlm_model, processor, image, prompt, device)
        # Store the result
        key = prompt.replace("What is the ", "").replace("?", "").strip()
        results[key] = prediction

    # Create a concise description
    concise_desc = f"given image: {results.get('intended usage', '').lower()} " \
                  f"{results.get('base color', '').lower()} " \
                  f"{results.get('type of article', '').lower()} " \
                  f"{results.get('subcategory', '').lower()} " \
                  f"for {results.get('season', '').lower()}"

    return concise_desc, results

def generate_vlm_prediction(model, processor, image, prompt, device):
    """Generate prediction for an image using VLM"""
    # Process image and text with the processor
    if isinstance(image, np.ndarray):
        # Convert OpenCV BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(image_rgb)
    else:
        pil_image = image

    # Process inputs
    model_inputs = processor(text=[prompt], images=[pil_image])
    model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

    # Get model inputs
    input_ids = model_inputs["input_ids"]
    attention_mask = model_inputs["attention_mask"]
    pixel_values = model_inputs["pixel_values"]

    # Import KVCache here to avoid circular imports
    from paligemma import KVCache
    kv_cache = KVCache()
    stop_token = processor.tokenizer.eos_token_id
    generated_tokens = []

    with torch.no_grad():
        for _ in range(50):  # Max 50 tokens for VLM output
            # Forward pass
            outputs = model(
                input_ids=input_ids,
                pixel_values=pixel_values,
                attention_mask=attention_mask,
                kv_cache=kv_cache,
            )
            kv_cache = outputs["kv_cache"]
            next_token_logits = outputs["logits"][:, -1, :]
            next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)
            token_id = next_token.item()
            generated_tokens.append(token_id)
            if token_id == stop_token:
                break
            input_ids = next_token
            attention_mask = torch.cat([attention_mask, torch.ones((1, 1), device=device)], dim=-1)

    generated_text = processor.tokenizer.decode(generated_tokens, skip_special_tokens=True)
    return generated_text

def format_input(instruction, input_text="", consider_wardrobe="no", consider_user_details="no", user_profile=None):
    """Format the instruction and input according to the training format, including personalization options."""
    # Base instruction
    instruction_text = (
        f"Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request. "
        f"Do not generate additional inputs or profile information."
        f"\n\n### Instruction:\n{instruction}"
    )

    # Add personalization details if requested
    if user_profile and consider_user_details.lower() == "yes":
        instruction_text += f"\n\nConsider the following user details for personalization:\n{user_profile.get_user_profile_summary()}"

    # Add wardrobe information if requested
    if user_profile and consider_wardrobe.lower() == "yes":
        instruction_text += f"\n\nConsider these items from the user's wardrobe for your suggestions:\n{user_profile.get_wardrobe_summary()}"

    # Add input text if provided
    input_part = f"\n\n### Input:\n{input_text}" if input_text else ""

    return instruction_text + input_part + "\n\n### Response:"

def generate_llm_text(model, tokenizer, prompt, device, max_new_tokens=150, temperature=0.7, top_k=40):
    """Generate text based on a prompt using the fine-tuned LLM."""
    # Tokenize the prompt
    encoded = torch.tensor(tokenizer.encode(prompt, allowed_special={"<|endoftext|>"})).unsqueeze(0).to(device)
    context_size = model.pos_emb.weight.shape[0]

    # Generate response
    model.eval()
    generated_tokens = []

    with torch.no_grad():
        for _ in range(max_new_tokens):
            # Get context for the current step
            idx_cond = encoded[:, -context_size:]

            # Forward pass
            logits = model(idx_cond)

            # Get logits for the last token
            logits = logits[:, -1, :]

            # Apply temperature and top-k filtering
            if temperature > 0:
                if top_k is not None:
                    v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                    logits[logits < v[:, [-1]]] = float('-inf')

                probs = torch.softmax(logits / temperature, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
            else:
                # Greedy sampling
                idx_next = torch.argmax(logits, dim=-1, keepdim=True)

            # Append new token
            encoded = torch.cat((encoded, idx_next), dim=1)
            generated_tokens.append(idx_next.item())

            # Stop if EOS token is generated (50256 is <|endoftext|>)
            if idx_next.item() == 50256:
                break

            # Also stop if we see "### Input:" - this is a sign the model is hallucinating additional input
            current_text = tokenizer.decode(encoded.squeeze().tolist()[len(tokenizer.encode(prompt)):])
            if "### Input:" in current_text:
                break

    # Decode the generated text
    generated_text = tokenizer.decode(encoded.squeeze().tolist())

    # Return only the response part (after the prompt)
    response = generated_text[len(prompt):]

    # Remove any hallucinated input sections
    if "### Input:" in response:
        response = response.split("### Input:")[0].strip()

    # Remove any hallucinated response markers
    if "### Response:" in response:
        response = response.replace("### Response:", "").strip()

    return response, generated_tokens


# ---------- WEBCAM CLASS ----------
class WebcamCapture:
    """Class to handle webcam capture functionality"""

    def __init__(self):
        self.cap = None
        self.is_running = False
        self.frame = None

    def start(self):
        """Start webcam capture"""
        if self.cap is None:
            try:
                self.cap = cv2.VideoCapture(0)
                if not self.cap.isOpened():
                    print("Failed to open webcam")
                    return False

                self.is_running = True
                return True
            except Exception as e:
                print(f"Error starting webcam: {e}")
                return False
        return True

    def stop(self):
        """Stop webcam capture"""
        if self.cap is not None:
            self.is_running = False
            self.cap.release()
            self.cap = None

    def get_frame(self):
        """Get current frame from webcam"""
        if self.cap is not None and self.cap.isOpened():
            ret, frame = self.cap.read()
            if ret:
                self.frame = frame
                return frame
        return None

    def capture_image(self):
        """Capture single image from webcam"""
        if self.frame is not None:
            return self.frame.copy()
        return self.get_frame()


# ---------- UI CLASSES ----------
class FashionSystemApp:
    """Main application class"""

    def __init__(self, root):
        self.root = root
        self.root.title("Fashion Recommendation System")
        self.root.geometry("1024x768")

        # Set theme
        style = ttk.Style()
        style.theme_use('clam')  # Use a modern-looking theme

        # Define colors
        self.bg_color = "#f0f0f0"
        self.accent_color = "#6c5ce7"
        self.text_color = "#2d3436"

        # Configure root window
        self.root.configure(bg=self.bg_color)

        # Initialize managers
        self.config_manager = ConfigManager(CONFIG_PATH)
        self.user_profile = UserProfileManager(USER_PROFILE_PATH)

        # Initialize webcam
        self.webcam = WebcamCapture()

        # Initialize model loading status
        self.models_loaded = False
        self.model_manager = None
        self.models = {}

        # Create UI elements
        self.create_widgets()

        # Start model loading in background thread
        self.start_model_loading()

    def create_widgets(self):
        """Create and configure all UI widgets"""
        # Create main container
        self.main_container = ttk.PanedWindow(self.root, orient=tk.HORIZONTAL)
        self.main_container.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        # Create sidebar frame
        self.sidebar = ttk.Frame(self.main_container, width=200)
        self.main_container.add(self.sidebar, weight=1)

        # Create content frame
        self.content = ttk.Frame(self.main_container)
        self.main_container.add(self.content, weight=3)

        # Create status bar
        self.status_bar = ttk.Label(self.root, text="Ready", anchor=tk.W, relief=tk.SUNKEN)
        self.status_bar.pack(side=tk.BOTTOM, fill=tk.X)

        # Setup sidebar buttons
        self.create_sidebar()

        # Setup content area
        self.show_home_screen()

    def create_sidebar(self):
        """Create sidebar with menu buttons"""
        # App title
        ttk.Label(self.sidebar, text="Fashion AI", font=("Arial", 16, "bold")).pack(pady=10)

        # Menu buttons
        btn_style = {'width': 20, 'padding': 5}

        ttk.Button(self.sidebar, text="Home", command=self.show_home_screen, **btn_style).pack(pady=5)
        ttk.Button(self.sidebar, text="Image Analysis", command=self.show_image_analysis, **btn_style).pack(pady=5)
        ttk.Button(self.sidebar, text="Text Advice", command=self.show_text_advice, **btn_style).pack(pady=5)
        ttk.Button(self.sidebar, text="User Profile", command=self.show_profile_editor, **btn_style).pack(pady=5)
        ttk.Button(self.sidebar, text="Settings", command=self.show_settings, **btn_style).pack(pady=5)
        ttk.Button(self.sidebar, text="Exit", command=self.on_exit, **btn_style).pack(pady=5)

        # Models status
        ttk.Separator(self.sidebar, orient=tk.HORIZONTAL).pack(fill=tk.X, pady=10)
        self.model_status_label = ttk.Label(self.sidebar, text="Models: Loading...", wraplength=180)
        self.model_status_label.pack(pady=5)

    def clear_content(self):
        """Clear all widgets from content frame"""
        for widget in self.content.winfo_children():
            widget.destroy()

    def show_home_screen(self):
        """Show home screen content"""
        self.clear_content()

        # Welcome message
        ttk.Label(self.content, text="Welcome to Fashion AI", font=("Arial", 24, "bold")).pack(pady=20)
        ttk.Label(self.content, text="Personalized fashion recommendations using AI", font=("Arial", 12)).pack()

        # Options frame
        options_frame = ttk.Frame(self.content)
        options_frame.pack(pady=40, expand=True)

        # Option buttons
        ttk.Button(options_frame, text="Analyze Fashion Image",
                  command=self.show_image_analysis, width=25, padding=10).pack(pady=10)
        ttk.Button(options_frame, text="Get Fashion Advice",
                  command=self.show_text_advice, width=25, padding=10).pack(pady=10)
        ttk.Button(options_frame, text="Edit User Profile",
                  command=self.show_profile_editor, width=25, padding=10).pack(pady=10)

    def show_image_analysis(self):
        """Show image analysis screen"""
        self.clear_content()

        if not self.models_loaded:
            ttk.Label(self.content, text="Models are still loading. Please wait...",
                    font=("Arial", 14)).pack(pady=20)
            return

        # Title
        ttk.Label(self.content, text="Fashion Image Analysis", font=("Arial", 18, "bold")).pack(pady=10)

        # Image selection frame
        img_frame = ttk.Frame(self.content)
        img_frame.pack(pady=10, fill=tk.X)

        # Image source buttons
        ttk.Button(img_frame, text="Use Webcam", command=self.use_webcam).pack(side=tk.LEFT, padx=5)
        ttk.Button(img_frame, text="Upload Image", command=self.upload_image).pack(side=tk.LEFT, padx=5)

        # Image display area
        self.image_display = ttk.Label(self.content)
        self.image_display.pack(pady=10)
        ttk.Label(self.image_display, text="No image selected").pack()

        # Image analysis results
        results_frame = ttk.LabelFrame(self.content, text="Analysis Results")
        results_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        # Results text area
        self.results_text = scrolledtext.ScrolledText(results_frame, height=10)
        self.results_text.pack(pady=5, fill=tk.BOTH, expand=True)

        # Query frame
        query_frame = ttk.Frame(self.content)
        query_frame.pack(pady=10, fill=tk.X)

        ttk.Label(query_frame, text="Ask about this fashion item:").pack(side=tk.LEFT, padx=5)
        self.query_entry = ttk.Entry(query_frame, width=50)
        self.query_entry.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)
        self.query_entry.insert(0, "What outfits can I create with this?")

        # Personalization options
        options_frame = ttk.Frame(self.content)
        options_frame.pack(pady=5, fill=tk.X)

        # Checkboxes for personalization
        self.use_wardrobe_var = tk.StringVar(value="no")
        ttk.Checkbutton(options_frame, text="Consider my wardrobe",
                       variable=self.use_wardrobe_var,
                       onvalue="yes", offvalue="no").pack(side=tk.LEFT, padx=5)

        self.use_profile_var = tk.StringVar(value="no")
        ttk.Checkbutton(options_frame, text="Consider my profile details",
                       variable=self.use_profile_var,
                       onvalue="yes", offvalue="no").pack(side=tk.LEFT, padx=5)

        # Generate button
        ttk.Button(self.content, text="Generate Recommendation",
                  command=self.generate_image_recommendation).pack(pady=10)

        # Response area
        response_frame = ttk.LabelFrame(self.content, text="Fashion Recommendation")
        response_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        self.response_text = scrolledtext.ScrolledText(response_frame, height=10)
        self.response_text.pack(pady=5, fill=tk.BOTH, expand=True)

    def show_text_advice(self):
        """Show text advice screen"""
        self.clear_content()

        if not self.models_loaded:
            ttk.Label(self.content, text="Models are still loading. Please wait...",
                    font=("Arial", 14)).pack(pady=20)
            return

        # Title
        ttk.Label(self.content, text="Fashion Advice", font=("Arial", 18, "bold")).pack(pady=10)

        # Instruction frame
        instruction_frame = ttk.Frame(self.content)
        instruction_frame.pack(pady=10, fill=tk.X)

        ttk.Label(instruction_frame, text="What fashion advice do you need?").pack(anchor=tk.W)

        # Query text area
        self.advice_query = scrolledtext.ScrolledText(instruction_frame, height=5)
        self.advice_query.pack(pady=5, fill=tk.BOTH)
        self.advice_query.insert(tk.END, "Suggest a professional outfit for a job interview")

        # Personalization options
        options_frame = ttk.Frame(self.content)
        options_frame.pack(pady=5, fill=tk.X)

        # Checkboxes for personalization
        self.text_use_wardrobe_var = tk.StringVar(value="no")
        ttk.Checkbutton(options_frame, text="Consider my wardrobe",
                       variable=self.text_use_wardrobe_var,
                       onvalue="yes", offvalue="no").pack(side=tk.LEFT, padx=5)

        self.text_use_profile_var = tk.StringVar(value="no")
        ttk.Checkbutton(options_frame, text="Consider my profile details",
                       variable=self.text_use_profile_var,
                       onvalue="yes", offvalue="no").pack(side=tk.LEFT, padx=5)

        # Generate button
        ttk.Button(self.content, text="Generate Fashion Advice",
                  command=self.generate_text_advice).pack(pady=10)

        # Response area
        response_frame = ttk.LabelFrame(self.content, text="Fashion Recommendation")
        response_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        self.text_response = scrolledtext.ScrolledText(response_frame, height=15)
        self.text_response.pack(pady=5, fill=tk.BOTH, expand=True)

    def show_profile_editor(self):
        """Show user profile editor screen"""
        self.clear_content()

        # Title
        ttk.Label(self.content, text="User Profile Editor", font=("Arial", 18, "bold")).pack(pady=10)

        # Profile data frame
        profile_frame = ttk.Frame(self.content)
        profile_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        # Create entry fields for user profile data
        row = 0
        self.profile_entries = {}

        # Basic profile fields
        basic_fields = ["name", "age", "gender", "height", "weight", "style_preference", "favorite_color"]

        for field in basic_fields:
            ttk.Label(profile_frame, text=field.replace('_', ' ').title() + ":").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)

            # Get value from user profile if available
            value = self.user_profile.user_data.get(field, "")

            entry = ttk.Entry(profile_frame, width=40)
            entry.insert(0, value)
            entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

            self.profile_entries[field] = entry
            row += 1

        # Wardrobe section
        ttk.Label(profile_frame, text="Wardrobe Items:").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)

        # Join wardrobe items with commas
        wardrobe_text = ", ".join(self.user_profile.user_wardrobe) if self.user_profile.user_wardrobe else ""

        self.wardrobe_entry = scrolledtext.ScrolledText(profile_frame, width=40, height=5)
        self.wardrobe_entry.insert(tk.END, wardrobe_text)
        self.wardrobe_entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)

        # Help text
        ttk.Label(profile_frame, text="Enter wardrobe items separated by commas").grid(row=row+1, column=1, sticky=tk.W, padx=5)

        # Save button
        ttk.Button(self.content, text="Save Profile", command=self.save_user_profile).pack(pady=20)

    def show_settings(self):
        """Show settings screen"""
        self.clear_content()

        # Title
        ttk.Label(self.content, text="Settings", font=("Arial", 18, "bold")).pack(pady=10)

        # Settings frame
        settings_frame = ttk.Frame(self.content)
        settings_frame.pack(pady=10, fill=tk.BOTH)

        # Get current config
        config = self.config_manager.get_config()

        # Create entries for numeric settings
        row = 0
        self.setting_entries = {}

        # Numeric settings
        ttk.Label(settings_frame, text="Max Tokens:").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        max_tokens_entry = ttk.Entry(settings_frame, width=10)
        max_tokens_entry.insert(0, str(config.get("max_new_tokens", 150)))
        max_tokens_entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
        self.setting_entries["max_new_tokens"] = max_tokens_entry
        ttk.Label(settings_frame, text="Max number of tokens to generate").grid(row=row, column=2, sticky=tk.W, padx=5)
        row += 1

        ttk.Label(settings_frame, text="Temperature:").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        temp_entry = ttk.Entry(settings_frame, width=10)
        temp_entry.insert(0, str(config.get("temperature", 0.7)))
        temp_entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
        self.setting_entries["temperature"] = temp_entry
        ttk.Label(settings_frame, text="Controls randomness (0.0-1.0)").grid(row=row, column=2, sticky=tk.W, padx=5)
        row += 1

        ttk.Label(settings_frame, text="Top K:").grid(row=row, column=0, sticky=tk.W, padx=5, pady=5)
        top_k_entry = ttk.Entry(settings_frame, width=10)
        top_k_entry.insert(0, str(config.get("top_k", 40)))
        top_k_entry.grid(row=row, column=1, sticky=tk.W, padx=5, pady=5)
        self.setting_entries["top_k"] = top_k_entry
        ttk.Label(settings_frame, text="Number of highest probability tokens to consider").grid(row=row, column=2, sticky=tk.W, padx=5)
        row += 1

        # Save button
        ttk.Button(self.content, text="Save Settings", command=self.save_settings).pack(pady=20)

        # Model information
        info_frame = ttk.LabelFrame(self.content, text="System Information")
        info_frame.pack(pady=10, fill=tk.BOTH, expand=True)

        # System info text
        info_text = f"""
        System Information:
        - Device: {torch.device("cuda" if torch.cuda.is_available() else "cpu")}
        - CUDA Available: {"Yes" if torch.cuda.is_available() else "No"}
        - Models Loaded: {"Yes" if self.models_loaded else "No"}
        """

        info_label = ttk.Label(info_frame, text=info_text, justify=tk.LEFT)
        info_label.pack(padx=10, pady=10, anchor=tk.W)

    def start_model_loading(self):
        """Start loading models in background thread"""
        self.update_status("Initializing model manager...")

        # Create model manager with status callback
        self.model_manager = ModelManager(status_callback=self.update_status)

        # Start loading in background thread
        loading_thread = threading.Thread(target=self.load_models_thread)
        loading_thread.daemon = True
        loading_thread.start()

    def load_models_thread(self):
        """Thread function to load models"""
        try:
            success = self.model_manager.initialize_models()
            if success:
                self.models = self.model_manager.get_models()
                self.models_loaded = True
                self.update_status("All models loaded successfully!")
                self.update_model_status("Models: Ready")
            else:
                self.update_status("Failed to load models.")
                self.update_model_status("Models: Error loading")
        except Exception as e:
            self.update_status(f"Error loading models: {str(e)}")
            self.update_model_status("Models: Error loading")

    def update_status(self, message):
        """Update status bar with message"""
        self.root.after(0, lambda: self.status_bar.config(text=message))

    def update_model_status(self, message):
        """Update model status label"""
        self.root.after(0, lambda: self.model_status_label.config(text=message))

    def upload_image(self):
        """Open file dialog to select an image"""
        if not self.models_loaded:
            messagebox.showinfo("Models Loading", "Please wait for models to finish loading")
            return

        file_path = filedialog.askopenfilename(
            title="Select Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )

        if file_path:
            try:
                # Load image
                img = cv2.imread(file_path)
                if img is None:
                    messagebox.showerror("Error", "Could not read image file")
                    return

                # Process and display image
                self.display_and_analyze_image(img)
            except Exception as e:
                messagebox.showerror("Error", f"Error loading image: {str(e)}")

    def use_webcam(self):
        """Use webcam to capture image"""
        if not self.models_loaded:
            messagebox.showinfo("Models Loading", "Please wait for models to finish loading")
            return

        # Initialize webcam if needed
        if not self.webcam.start():
            messagebox.showerror("Error", "Could not initialize webcam")
            return

        # Create webcam capture window
        webcam_window = tk.Toplevel(self.root)
        webcam_window.title("Webcam Capture")
        webcam_window.geometry("640x520")

        # Create frame for video
        video_frame = ttk.Frame(webcam_window)
        video_frame.pack(pady=10)

        # Create label for video
        video_label = ttk.Label(video_frame)
        video_label.pack()

        # Create capture button
        ttk.Button(webcam_window, text="Capture", command=lambda: self.capture_webcam_image(webcam_window)).pack(pady=10)

        # Update webcam frame
        self.update_webcam_frame(video_label, webcam_window)

    def update_webcam_frame(self, label, window):
        """Update webcam frame in UI"""
        if not hasattr(window, 'closed') or not window.closed:
            frame = self.webcam.get_frame()
            if frame is not None:
                # Convert to RGB for PIL
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                img = Image.fromarray(frame_rgb)
                img = ImageTk.PhotoImage(image=img)

                # Update label
                label.configure(image=img)
                label.image = img

            # Schedule next update
            window.after(30, lambda: self.update_webcam_frame(label, window))

    def capture_webcam_image(self, window):
        """Capture image from webcam and close window"""
        img = self.webcam.capture_image()
        if img is not None:
            # Stop webcam
            self.webcam.stop()

            # Close window
            window.closed = True
            window.destroy()

            # Process and display image
            self.display_and_analyze_image(img)
        else:
            messagebox.showerror("Error", "Could not capture image from webcam")

    def display_and_analyze_image(self, img):
        """Display image and analyze with VLM"""
        try:
            # Resize for display
            display_img = cv2.resize(img, (300, 300))

            # Convert to RGB for PIL
            display_img_rgb = cv2.cvtColor(display_img, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(display_img_rgb)
            photo_img = ImageTk.PhotoImage(image=pil_img)

            # Update image display
            self.clear_widget(self.image_display)
            self.image_display.configure(image=photo_img)
            self.image_display.image = photo_img

            # Process with VLM
            self.update_status("Analyzing image with VLM...")

            # Get models
            vlm_model = self.models.get("vlm")
            processor = self.models.get("processor")
            device = self.models.get("device")

            # Process image
            concise_desc, results = process_image_with_vlm(vlm_model, processor, img, device)

            # Display results
            self.results_text.delete(1.0, tk.END)
            self.results_text.insert(tk.END, f"Image Description: {concise_desc}\n\n")

            for key, value in results.items():
                self.results_text.insert(tk.END, f"{key.title()}: {value}\n")

            self.update_status("Image analysis complete!")
        except Exception as e:
            self.update_status(f"Error processing image: {str(e)}")
            messagebox.showerror("Error", f"Error processing image: {str(e)}")

    def generate_image_recommendation(self):
        """Generate fashion recommendation based on image and query"""
        if not self.models_loaded:
            messagebox.showinfo("Models Loading", "Please wait for models to finish loading")
            return

        # Check if image is loaded
        if not hasattr(self.image_display, 'image') or self.image_display.image is None:
            messagebox.showinfo("No Image", "Please upload or capture an image first")
            return

        # Get query
        query = self.query_entry.get().strip()
        if not query:
            messagebox.showinfo("Empty Query", "Please enter a question about the fashion item")
            return

        # Get personalization options
        consider_wardrobe = self.use_wardrobe_var.get()
        consider_user_details = self.use_profile_var.get()

        # Get image description from results
        image_desc = self.results_text.get(1.0, tk.END).strip()

        # Prepare input for LLM
        instruction = f"Provide fashion advice about the following clothing item: {query}"
        input_text = f"The image shows: {image_desc}"

        # Format input with personalization options
        formatted_input = format_input(
            instruction,
            input_text,
            consider_wardrobe,
            consider_user_details,
            self.user_profile
        )

        # Generate response
        self.update_status("Generating recommendation...")
        try:
            # Get models
            llm_model = self.models.get("llm")
            tokenizer = self.models.get("tokenizer")
            device = self.models.get("device")

            # Get generation parameters
            config = self.config_manager.get_config()
            max_tokens = int(config.get("max_new_tokens", 150))
            temperature = float(config.get("temperature", 0.7))
            top_k = int(config.get("top_k", 40))

            # Generate text
            response, _ = generate_llm_text(
                llm_model,
                tokenizer,
                formatted_input,
                device,
                max_tokens,
                temperature,
                top_k
            )

            # Display response
            self.response_text.delete(1.0, tk.END)
            self.response_text.insert(tk.END, response)

            self.update_status("Recommendation generated!")
        except Exception as e:
            self.update_status(f"Error generating recommendation: {str(e)}")
            messagebox.showerror("Error", f"Error generating recommendation: {str(e)}")

    def generate_text_advice(self):
        """Generate fashion advice based on text query"""
        if not self.models_loaded:
            messagebox.showinfo("Models Loading", "Please wait for models to finish loading")
            return

        # Get query
        query = self.advice_query.get(1.0, tk.END).strip()
        if not query:
            messagebox.showinfo("Empty Query", "Please enter a fashion advice question")
            return

        # Get personalization options
        consider_wardrobe = self.text_use_wardrobe_var.get()
        consider_user_details = self.text_use_profile_var.get()

        # Prepare input for LLM
        instruction = f"Provide fashion advice: {query}"

        # Format input with personalization options
        formatted_input = format_input(
            instruction,
            "",
            consider_wardrobe,
            consider_user_details,
            self.user_profile
        )

        # Generate response
        self.update_status("Generating fashion advice...")
        try:
            # Get models
            llm_model = self.models.get("llm")
            tokenizer = self.models.get("tokenizer")
            device = self.models.get("device")

            # Get generation parameters
            config = self.config_manager.get_config()
            max_tokens = int(config.get("max_new_tokens", 150))
            temperature = float(config.get("temperature", 0.7))
            top_k = int(config.get("top_k", 40))

            # Generate text
            response, _ = generate_llm_text(
                llm_model,
                tokenizer,
                formatted_input,
                device,
                max_tokens,
                temperature,
                top_k
            )

            # Display response
            self.text_response.delete(1.0, tk.END)
            self.text_response.insert(tk.END, response)

            self.update_status("Fashion advice generated!")
        except Exception as e:
            self.update_status(f"Error generating advice: {str(e)}")
            messagebox.showerror("Error", f"Error generating advice: {str(e)}")

    def save_user_profile(self):
        """Save user profile data"""
        try:
            # Collect data from entry fields
            profile_data = {}

            # Get basic profile fields
            for field, entry in self.profile_entries.items():
                profile_data[field] = entry.get().strip()

            # Get wardrobe items
            wardrobe_text = self.wardrobe_entry.get(1.0, tk.END).strip()
            wardrobe_items = [item.strip() for item in wardrobe_text.split(',') if item.strip()]
            profile_data['wardrobe'] = wardrobe_items

            # Save profile
            if self.user_profile.save_profile(profile_data):
                messagebox.showinfo("Success", "User profile saved successfully")
                self.update_status("User profile saved")
            else:
                messagebox.showerror("Error", "Failed to save user profile")
        except Exception as e:
            messagebox.showerror("Error", f"Error saving profile: {str(e)}")

    def save_settings(self):
        """Save application settings"""
        try:
            # Collect settings from entry fields
            new_config = {}

            # Get numeric settings
            for setting, entry in self.setting_entries.items():
                value = entry.get().strip()

                # Convert to appropriate type
                if setting in ["max_new_tokens", "top_k"]:
                    value = int(value)
                elif setting == "temperature":
                    value = float(value)

                new_config[setting] = value

            # Update config
            if self.config_manager.update_config(new_config):
                messagebox.showinfo("Success", "Settings saved successfully")
                self.update_status("Settings saved")
            else:
                messagebox.showerror("Error", "Failed to save settings")
        except ValueError:
            messagebox.showerror("Error", "Please enter valid numeric values for settings")
        except Exception as e:
            messagebox.showerror("Error", f"Error saving settings: {str(e)}")

    def clear_widget(self, widget):
        """Clear a widget's content"""
        if isinstance(widget, ttk.Label):
            widget.configure(image='')
            if hasattr(widget, 'image'):
                delattr(widget, 'image')
        elif isinstance(widget, scrolledtext.ScrolledText):
            widget.delete(1.0, tk.END)
        elif isinstance(widget, ttk.Entry):
            widget.delete(0, tk.END)

    def on_exit(self):
        """Handle application exit"""
        if messagebox.askokcancel("Exit", "Are you sure you want to exit?"):
            # Stop webcam if running
            if self.webcam:
                self.webcam.stop()

            # Destroy root window
            self.root.destroy()


# ---------- MAIN ENTRY POINT ----------
def main():
    """Main entry point for the application"""
    # Create root window
    # Install xvfb if needed (in Colab)
    !apt-get install -y xvfb

    # Set up virtual display
    !Xvfb :1 -screen 0 1024x768x24 &
    import os
    os.environ['DISPLAY'] = ':1'

    root = tk.Tk()

    # Create app instance
    app = FashionSystemApp(root)

    # Start main loop
    root.mainloop()

if __name__ == "__main__":
    main()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.14).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


TclError: couldn't connect to display ":1"